# Parsing Shipping Data

<h2 id="tocheading">Table of Contents</h2>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [7]:
# I often switch between editing notebooks from my PC to my Mac;
# the code in this cell is to find the proper local data when I do so.

import platform
import os
import locale
import json

o_s = platform.system()
paths = {}

if o_s == 'Darwin':
    paths['data_dir'] = '/Users/drew/data/Data_Aggregation'
elif o_s == 'Windows':
    paths['data_dir'] = r'C:\Users\DMacKellar\Documents\Data\Data_Aggregation'

locale.setlocale(locale.LC_ALL, '')

'en_US.UTF-8'

## Introduction

I was recently playing with Pandas' built-in DataReader module in a project to explore some financial markets' data.  Many online examples involve using it to draw historical stock price and volume data from Yahoo or Google Finance.  Yahoo's financial data is now completely defunct (not accessible via the pandas module; there is still a [web interface](https://finance.yahoo.com/)), and a warning has been added that the Google Finance data interface is prone to failure (and, indeed, I haven't been able to recover data with it, either).  So I was perusing [the docs](https://pandas-datareader.readthedocs.io/en/latest/) associated with the DataReader module.

While looking over the other data sources listed in the Pandas Datareader docs page, I saw one listed as "the world’s largest repository of structured public data", referring to a site called [Enigma](https://public.enigma.com/).  It does indeed have some interesting and diverse sets of data, and I quickly settled on [Bills of Lading](https://public.enigma.com/spotlight/bill-of-lading) from among the links on the front page, a set which claimed to list the contents of tons of shipping from around the world.  The top few rows listed today happened to have Seattle, WA as the port of unlading, and I thought it would be a cool practice to try to categorize all of the arrivals in Seattle for a period, and how they change over time.  [Their API](http://docs.enigma.com/public/public_v20_user_api.html) involves installing a program called Postman; I downloaded it and set it up via my Google account.  I also signed up for Enigma using my Gmail account, so it appears to have transferred the credentials automatically, because all I had to do to open the API was copy the URL from Enigma's page (after clicking on the API button within the dataset's window), paste it into the top field in the Postman window, and it returned the basic data.

I'm accessing this on 20180408, and the entire Bills of Lading dataset states that it has 27,788,013 rows.  But the date range is just for 2017.  They also have separate sets for each year going back to 2014.

To be specific, the overall organization of this dataset within Enigma's system is:

    International Trade & Shipping
    -> United States Import Records

But within that they have 3 different sections:
       
       -> United States Import Records - Bill of Lading Summary
          United States Import Records - Cargo
          United States Import Records - Toxic & Hazardous Materials
          
The first two have results for each year 2014-2017, but the toxic materials data just has 2015.

Summaries for each of these are offered in the "overview" tab:


*Bill of Lading Summary*:

Bills of lading header information for incoming shipments regulated by U.S. Customs and Border Protection's Automated Manifest System (AMS) for 2017.
Updated a month ago
27,788,013 rows
31 fields

*Cargo Summary*:

Cargo imports in 2017. The Automated Manifest System (AMS) is designed by U.S. Customs to facilitate cargo information between steam ship lines, airlines and rail carriers for shipments destined to or transiting the United States. Recently, the processing of other cargo transportation (electronic truck, rail and sea) manifests has been transitioned to Automated Commercial Environment (ACE). AMS only refers to electronic air manifests.
Read less
Updated 19 days ago
32,861,196 rows
9 fields

*Toxic & Hazardous Materials*:

Information for incoming shipments containing toxic and hazardous materials provided by U.S. Customs and Border Protection's Automated Manifest System (AMS) and Automated Commercial Environment (ACE).
Read less
Updated 9 months ago

---

Actually, after poking around for a bit, using their default browser-based data viewer is pretty handy, too.  Click on the leftmost icon that appears when hovering over any column name, and [you can enter specific strings to filter its content](http://docs.enigma.com/public/public_v20_user_filtering.html).  When I enter "Seattle, Washington" for Port of Unlading, it returns 27,100 of 27,788,013 rows in the Bill of Lading Summary - 2017 dataset.  There's then a button to export the resulting, filtered data set as a csv file, which in this case is ~14MB.  Import below:

In [3]:
import platform
import numpy as np
import pandas as pd
import multiprocessing as mp
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import os

o_s = platform.system()
paths = {}

if o_s == 'Darwin':
    paths['enigma_dir'] = '/Users/drew/data/Enigma'
elif o_s == 'Windows':
    paths['enigma_dir'] = r'C:\Users\DMacKellar\Documents\Data\Enigma'

paths['sea2017'] = os.path.join(paths['enigma_dir'], 
                                'seattle_shipping_2017.csv')

sea2017_df = pd.read_csv(paths['sea2017'])

In [9]:
sea2017_df.head()

,identifier,trade_update_date,run_date,vessel_name,port_of_unlading,estimated_arrival_date,foreign_port_of_lading,record_status_indicator,place_of_receipt,port_of_destination,...,shipper_comm_number_qualifier,shipper_comm_number,container_number,description_sequence_number,piece_count,description_text,harmonized_number,harmonized_value,harmonized_weight,harmonized_weight_unit
0,201712261102,2017-11-22T00:00:00,2017-12-26T00:00:00,NEW YORK EXPRESS,"Seattle, Washington",2017-12-21T00:00:00,"Hong Kong,Hong Kong",New,NANSHA CHINA,NaN,...,NaN,NaN,TCNU3666378,1,127,"SOFA & UPHOLSTERY PO418287/418285/418967 ""SH """,NaN,NaN,NaN,NaN
1,201712261102,2017-11-22T00:00:00,2017-12-26T00:00:00,NEW YORK EXPRESS,"Seattle, Washington",2017-12-21T00:00:00,"Hong Kong,Hong Kong",New,NANSHA CHINA,NaN,...,NaN,NaN,YMLU8500198,1,136,"SOFA & UPHOLSTERY PO418287/418285/418967 ""SH """,NaN,NaN,NaN,NaN
2,201712261102,2017-11-22T00:00:00,2017-12-26T00:00:00,NEW YORK EXPRESS,"Seattle, Washington",2017-12-21T00:00:00,"Hong Kong,Hong Kong",New,NANSHA CHINA,NaN,...,NaN,NaN,YMLU8687140,1,56,"SOFA & UPHOLSTERY PO418287/418285/418967 ""SH """,NaN,NaN,NaN,NaN
3,201712261237,2017-11-23T00:00:00,2017-12-26T00:00:00,YM UNANIMITY,"Seattle, Washington",2017-12-20T00:00:00,"Tsingtao,China (Mainland)",New,QINGDAO,NaN,...,NaN,NaN,UACU5040364,1,1263,PE TARPAULIN THIS SHIPMENT CONTAINS NO WOOD P ...,NaN,NaN,NaN,NaN
4,201712261259,2017-11-29T00:00:00,2017-12-26T00:00:00,NEW YORK EXPRESS,"Seattle, Washington",2017-12-21T00:00:00,"Pusan,South Korea",New,DALIAN CHINA,NaN,...,NaN,NaN,YMLU8721007,1,37,WOODEN DOORS,NaN,NaN,NaN,NaN


In [10]:
sea2017_df.describe()

,identifier,foreign_port_of_destination,description_sequence_number,piece_count,harmonized_number,harmonized_value,harmonized_weight
count,2.710000e+04,0.0,27100.000000,27100.000000,1.597600e+04,1.643600e+04,16436.000000
mean,1.018439e+12,NaN,2.278155,355.122509,3.327146e+07,2.503881e+05,11218.306461
std,9.064852e+11,NaN,8.894217,1087.079547,4.898259e+08,7.272369e+05,28254.145238
min,2.017123e+10,NaN,1.000000,1.000000,2.013000e+04,0.000000e+00,0.000000
25%,2.017123e+11,NaN,1.000000,8.000000,2.931900e+05,0.000000e+00,0.000000
50%,2.017123e+11,NaN,1.000000,26.000000,4.409100e+05,0.000000e+00,0.000000
75%,2.017123e+12,NaN,1.000000,393.250000,8.422900e+05,4.243400e+05,23885.750000
max,2.017123e+12,NaN,133.000000,101134.000000,9.404210e+09,9.748180e+06,487409.000000


## Counting Cities

I'll start by just enumerating the sources of the material imported into our fair city (as of 2017, according to this data set):

In [11]:
sea2017_df['foreign_port_of_lading'].value_counts()[:15]

Vancouver, BC ,Canada         13102
Pusan,South Korea              3240
Shanghai ,China (Mainland)     2936
Ningpo ,China (Mainland)       1999
Yantian,China (Mainland)        999
Hong Kong,Hong Kong             940
Xiamen,China (Mainland)         777
57037                           721
Sines,Portugal                  373
Auckland,New Zealand            317
Manzanillo,Mexico               266
Cartagena ,Colombia             181
Puerto Quetzal ,Guatemala       148
Kaohsiung,China (Taiwan)        142
La Spezia,Italy                 121
Name: foreign_port_of_lading, dtype: int64

The counts of numbers of entries in the table originating in each city might be interesting (and relatively straightforward) to visualize.  First, I'll need to clean up and reformat the city names.

To begin with, it's apparent from the list above that some of the ports of lading are just numerical codes.  By Googling, I find [resources](https://www.cbp.gov/sites/default/files/assets/documents/2017-Feb/appendix_f_0.pdf) that can be used to translate them.  For instance, 57037 is Yangshan, China.  I wonder how many such values there are.

In [12]:
matches = sea2017_df['foreign_port_of_lading'].str.match(r'[\d.]{5}')
sea2017_df.loc[matches, 'foreign_port_of_lading'].value_counts()

57037    721
57073      2
35136      1
Name: foreign_port_of_lading, dtype: int64

Ok, just 3 unique entries.  That should be quick to fix.  Except that I can't seem to find any city corresponding to 35136.  It's probably an error in the data entry.

In [95]:
origin_codes_dict = {'57037': 'Yangshan ,China (Mainland)', 
                     '57073': 'Nansha ,China (Mainland)',
                     '35136': np.nan
                    }

sea2017_df['foreign_port_of_lading'].replace(origin_codes_dict, 
                                             inplace=True)

# No point keeping record with no city listed
sea2017_df = sea2017_df[sea2017_df['foreign_port_of_lading'].notnull()]

In [14]:
print(sea2017_df['foreign_port_of_lading'].describe())

count                     27099
unique                       77
top       Vancouver, BC ,Canada
freq                      13102
Name: foreign_port_of_lading, dtype: object


Now, let's see where we get our imports from.  I'll want to use [plotly](https://plot.ly/python/bubble-maps/) to make a bubble map in a global mercator projection.  To get the coordinates for the cities, I'd like to try programmatic access to [Google Geocoding](https://developers.google.com/maps/documentation/geocoding/start), provided via [geopy](https://pypi.python.org/pypi/geopy/1.9.1).  One question is whether the interface is smart enough to handle the messy strings provided in the data set, or whether I'll have to clean them up, first.

In [15]:
import plotly.plotly as plotly
from geopy.geocoders import Nominatim

geolocator = Nominatim()
location1 = geolocator.geocode("Kaohsiung,China (Taiwan)")
location2 = geolocator.geocode("Kaohsiung, China (Taiwan)")
location3 = geolocator.geocode("Kaohsiung, China")
locations = [location1, location2, location3]
for location in locations:
    if location:
        print(location, location.latitude, location.longitude)
    else:
        print('{} not found'.format(location))

None not found
None not found
中鋼, 鳳宮里, 大林蒲, 小港區, 高雄市, 812, 臺灣 22.5447429 120.356598755463


Ok, yeah; they need to be cleaned up.  We have cities always first, then a country, with a comma in between.  But we have some with no spaces on either side of the comma, with spaces on one side or the other, or on both sides, and none appear to be tolerated.  This definitely requires some regex.  Or maybe just '`.split()`' and '`.strip()`'.

In [16]:
import re

words = re.compile('[A-z]+')

def get_words(source):
    stuff = re.sub(r'\([^)]*\)', '', source)
    parts = stuff.split(sep=',')
    formatted = []
    for part in parts:
        new_parts = []
        part = part.strip()
        parts_words = re.findall(words, part)
        new_parts.append(' '.join(parts_words))
        formatted.append(', '.join(new_parts))
    return str(', '.join(formatted))

sea2017_df['lading_formatted'] = sea2017_df['foreign_port_of_lading']\
.astype(str).apply(get_words)

sea2017_df['lading_formatted'].value_counts()[:10]

Vancouver, BC, Canada    13102
Pusan, South Korea        3240
Shanghai, China           2936
Ningpo, China             1999
Yantian, China             999
Hong Kong, Hong Kong       940
Xiamen, China              777
Yangshan, China            721
Sines, Portugal            373
Auckland, New Zealand      317
Name: lading_formatted, dtype: int64

Man, that took forever; I kept trying to exclude the junk in parentheses with a regex exclude clause, and couldn't get it to work.  It turns out quicker to create a new variable using `re.sub` to just get rid of anything in parentheses.  Anyways, now I want to feed them to geolocator iteratively, and build a new column with any lat/long I get out.

I had some trouble with the default `Nominatim` geolocator, imported above, when trying to run the cell below.  It's specifically calling upon the [Openstreetmap server](http://geopy.readthedocs.io/en/latest/#geopy.geocoders.Nominatim), which leaves too many cities without coords.  Also, after trying to re-run them, I'm seeing timeout and other server-interaction errors.  So instead I ended up trying a couple others, and `ArcGIS` seems to work the best.

In [18]:
from geopy import geocoders

geolocator = geocoders.ArcGIS()

cities = sea2017_df['lading_formatted'].unique()
lat_long = dict()
failures = []

for city in cities:
    location = geolocator.geocode(city)
    if location:
        lat = location.latitude
        long = location.longitude
        lat_long[city] = (lat, long)
    else:
        failures.append(city)

In [19]:
from itertools import islice

list(lat_long.items())[:10]

[('Hong Kong, Hong Kong', (22.351958323000076, 114.1193859870001)),
 ('Tsingtao, China', (36.098610000000065, 120.37194000000011)),
 ('Pusan, South Korea', (35.10278000000005, 129.04028000000005)),
 ('Yantian, China', (26.853330000000028, 119.85750000000007)),
 ('Shanghai, China', (31.22222000000005, 121.45806000000005)),
 ('Ningpo, China', (29.86569000000003, 121.53916000000004)),
 ('Yangshan, China', (35.18333000000007, 116.25000000000011)),
 ('Vancouver, BC, Canada', (49.260380000000055, -123.11335999999994)),
 ('Sines, Portugal', (37.957250000000045, -8.86934999999994)),
 ('Yentai, China', (37.52803000000006, 121.3826600000001))]

In [20]:
print(len(lat_long.keys()), len(failures))

77 0


Using '`geolocator2 = geocoders.GoogleV3()`' yields:

    GeocoderQueryError: HTTP Error 400: Bad Request

Probably something outdated about the API/geocode interface.  I tried getting an API key, which worked, but it still returns '`Bad Request`'.

Anyways, I ended up trying [ArcGIS](https://developers.arcgis.com/rest/geocode/api-reference/overview-world-geocoding-service.htm), and that was sufficient to get the rest.

In [21]:
def lookup(city):
    if city in lat_long.keys():
        coords = lat_long[city]
    else:
        coords = np.nan
    return coords

sea2017_df['lat_long'] = sea2017_df['lading_formatted'].astype(str).apply(lookup)

In [22]:
print('Cities lacking lat-long: {0:.2f}%'.format(sea2017_df['lat_long'].isnull().sum() / sea2017_df.shape[1] * 100))
sea2017_df.head()

Cities lacking lat-long: 0.00%


,identifier,trade_update_date,run_date,vessel_name,port_of_unlading,estimated_arrival_date,foreign_port_of_lading,record_status_indicator,place_of_receipt,port_of_destination,...,container_number,description_sequence_number,piece_count,description_text,harmonized_number,harmonized_value,harmonized_weight,harmonized_weight_unit,lading_formatted,lat_long
0,201712261102,2017-11-22T00:00:00,2017-12-26T00:00:00,NEW YORK EXPRESS,"Seattle, Washington",2017-12-21T00:00:00,"Hong Kong,Hong Kong",New,NANSHA CHINA,NaN,...,TCNU3666378,1,127,"SOFA & UPHOLSTERY PO418287/418285/418967 ""SH """,NaN,NaN,NaN,NaN,"Hong Kong, Hong Kong","(22.351958323000076, 114.1193859870001)"
1,201712261102,2017-11-22T00:00:00,2017-12-26T00:00:00,NEW YORK EXPRESS,"Seattle, Washington",2017-12-21T00:00:00,"Hong Kong,Hong Kong",New,NANSHA CHINA,NaN,...,YMLU8500198,1,136,"SOFA & UPHOLSTERY PO418287/418285/418967 ""SH """,NaN,NaN,NaN,NaN,"Hong Kong, Hong Kong","(22.351958323000076, 114.1193859870001)"
2,201712261102,2017-11-22T00:00:00,2017-12-26T00:00:00,NEW YORK EXPRESS,"Seattle, Washington",2017-12-21T00:00:00,"Hong Kong,Hong Kong",New,NANSHA CHINA,NaN,...,YMLU8687140,1,56,"SOFA & UPHOLSTERY PO418287/418285/418967 ""SH """,NaN,NaN,NaN,NaN,"Hong Kong, Hong Kong","(22.351958323000076, 114.1193859870001)"
3,201712261237,2017-11-23T00:00:00,2017-12-26T00:00:00,YM UNANIMITY,"Seattle, Washington",2017-12-20T00:00:00,"Tsingtao,China (Mainland)",New,QINGDAO,NaN,...,UACU5040364,1,1263,PE TARPAULIN THIS SHIPMENT CONTAINS NO WOOD P ...,NaN,NaN,NaN,NaN,"Tsingtao, China","(36.098610000000065, 120.37194000000011)"
4,201712261259,2017-11-29T00:00:00,2017-12-26T00:00:00,NEW YORK EXPRESS,"Seattle, Washington",2017-12-21T00:00:00,"Pusan,South Korea",New,DALIAN CHINA,NaN,...,YMLU8721007,1,37,WOODEN DOORS,NaN,NaN,NaN,NaN,"Pusan, South Korea","(35.10278000000005, 129.04028000000005)"


Ok, even though ArcGIS worked, it is finicky, and requires multiple tries, timing out often.  Since I've gotten the coordinates for the cities in a useable df, let's save this, and reload it in the future.

In [23]:
paths['sea2017_cities_coords_pkl'] = \
os.path.join(paths['enigma_dir'], 'sea2017_cities_coords.pkl')

with open(paths['sea2017_cities_coords_pkl'], 'wb') as f:
    pickle.dump(sea2017_df, f)

In [24]:
paths['enigma_dir']

'/Users/drew/data/Enigma'

# ~~If Restarting the Notebook, Resume with Cell Below~~ (Outdated)

In [4]:
# I often switch between editing notebooks from my PC to my Mac;
# the code in this cell is to find the proper local data when I do so.

import platform
import os
import locale
import platform
import numpy as np
import pandas as pd
import multiprocessing as mp
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

o_s = platform.system()
paths = {}

if o_s == 'Darwin':
    paths['data_dir'] = '/Users/drew/data/Enigma'
elif o_s == 'Windows':
    paths['data_dir'] = r'C:\Users\DMacKellar\Documents\Data\
\Enigma'

locale.setlocale(locale.LC_ALL, '')

for root, dirs, files in os.walk(paths['data_dir']):
    for file in files:
        file_under = str(file).replace('.', '_')
        path = os.path.join(root, file)
        paths[file_under] = path
        
# That should populate all data files.  Whenever a loading call fails, do:
# print(*paths.keys(), sep='\n')
# And look for the key with the proper name (with underscore instead of period)

In [99]:
with open(paths['sea2017_cities_coords_pkl'], 'rb') as pickle_file:
    sea2017_df = pickle.load(pickle_file)
    
sea2017_df.head()

,identifier,trade_update_date,run_date,vessel_name,port_of_unlading,estimated_arrival_date,foreign_port_of_lading,record_status_indicator,place_of_receipt,port_of_destination,...,container_number,description_sequence_number,piece_count,description_text,harmonized_number,harmonized_value,harmonized_weight,harmonized_weight_unit,lading_formatted,lat_long
0,201712261102,2017-11-22T00:00:00,2017-12-26T00:00:00,NEW YORK EXPRESS,"Seattle, Washington",2017-12-21T00:00:00,"Hong Kong,Hong Kong",New,NANSHA CHINA,NaN,...,TCNU3666378,1,127,"SOFA & UPHOLSTERY PO418287/418285/418967 ""SH """,NaN,NaN,NaN,NaN,"Hong Kong, Hong Kong","(22.351958323000076, 114.1193859870001)"
1,201712261102,2017-11-22T00:00:00,2017-12-26T00:00:00,NEW YORK EXPRESS,"Seattle, Washington",2017-12-21T00:00:00,"Hong Kong,Hong Kong",New,NANSHA CHINA,NaN,...,YMLU8500198,1,136,"SOFA & UPHOLSTERY PO418287/418285/418967 ""SH """,NaN,NaN,NaN,NaN,"Hong Kong, Hong Kong","(22.351958323000076, 114.1193859870001)"
2,201712261102,2017-11-22T00:00:00,2017-12-26T00:00:00,NEW YORK EXPRESS,"Seattle, Washington",2017-12-21T00:00:00,"Hong Kong,Hong Kong",New,NANSHA CHINA,NaN,...,YMLU8687140,1,56,"SOFA & UPHOLSTERY PO418287/418285/418967 ""SH """,NaN,NaN,NaN,NaN,"Hong Kong, Hong Kong","(22.351958323000076, 114.1193859870001)"
3,201712261237,2017-11-23T00:00:00,2017-12-26T00:00:00,YM UNANIMITY,"Seattle, Washington",2017-12-20T00:00:00,"Tsingtao,China (Mainland)",New,QINGDAO,NaN,...,UACU5040364,1,1263,PE TARPAULIN THIS SHIPMENT CONTAINS NO WOOD P ...,NaN,NaN,NaN,NaN,"Tsingtao, China","(36.098610000000065, 120.37194000000011)"
4,201712261259,2017-11-29T00:00:00,2017-12-26T00:00:00,NEW YORK EXPRESS,"Seattle, Washington",2017-12-21T00:00:00,"Pusan,South Korea",New,DALIAN CHINA,NaN,...,YMLU8721007,1,37,WOODEN DOORS,NaN,NaN,NaN,NaN,"Pusan, South Korea","(35.10278000000005, 129.04028000000005)"


Now I want to count the total number of containers arriving based on the city of origin, and move them into a new dataframe indexed by city name, and including the coordinates of each.

In [100]:
city_coords = sea2017_df['lat_long'].groupby(
    sea2017_df['lading_formatted']).unique().rename('coords')

# The lat/long are trapped in unnecessary lists: bust them out
city_coords = city_coords.apply(lambda x: x[0])

city_counts = sea2017_df['lading_formatted'].value_counts().rename('containers')

city_records = pd.concat([city_coords, city_counts], axis=1)

# Actually, let's just make lat and long explicit columns
city_records['lat'] = city_records['coords'].apply(lambda x: (x)[0])
city_records['long'] = city_records['coords'].apply(lambda x: (x)[1])

In [101]:
city_records.head()

,coords,containers,lat,long
"Adelaide, Australia","(-34.92584999999997, 138.5998300000001)",13,-34.925850,138.599830
"All Other Oman Ports, Oman","(22.993080000000077, 72.52465000000007)",1,22.993080,72.524650
"Anvers, Belgium","(51.22209000000004, 4.397680000000037)",6,51.222090,4.397680
"Auckland, New Zealand","(-36.70917666799994, 174.7330887700001)",317,-36.709177,174.733089
"Balboa, Panama","(8.370532224000044, -78.95069304799995)",90,8.370532,-78.950693


Alright, let's try making the plot.  Since I don't know Plotly that well, I'll start by trying to import [a cell with an example from their docs](https://plot.ly/python/bubble-maps/) wholesale, then modify the most likely fields.  But I keep getting errors on the final line, referring to an error loading a URL or signing.  It appears that [Plotly wants you to get an API key](https://plot.ly/python/getting-started/#initialization-for-online-plotting), so I signed up and I'll store the key locally:

In [102]:
import json

# DCM code for retrieving credentials
paths['plotly_cred'] = os.path.join(paths['data_dir'], 'plotly_api_credentials.json')

with open(paths['plotly_cred'], 'r') as f:
    cred = json.load(f)
    
plotly_username = cred['username']
plotly_api_key = cred['api_key']

First, a bare-bones example to simply see if the map will render with the cities in their proper location:

In [103]:
import plotly.plotly as py

trace = dict(type='scattergeo',
             lon=city_records['long'].values,
             lat=city_records['lat'].values,
             text=city_records.index.values.tolist(),
             mode='markers')
py.iplot([trace])
# print(trace)

Ok, cool; that was easy.  Now to adapt the tutorial's code to get the more elaborate example, where you partition the data into groups of cities based on some variable.  In this case, I'll choose based on the number of shipments they made to Seattle in 2017.  I'd like a dynamic way to do this, so I'll choose the `pd.qcut` method:

In [104]:
city_records['containers'].quantile(np.round(np.linspace(0, 1, num=4))).values.tolist()

group_labels = list(((np.rint(x.left).astype(int), 
                      np.round(x.right).astype(int)) for x in pd.qcut(x=city_records['containers'], q=5, precision=0, duplicates='drop').values.unique().sort_values()))

group_labels
for i, interval in enumerate(pd.qcut(x=city_records['containers'], q=3, precision=0).values.unique().sort_values()):
    print(i, interval)

0 (0.0, 2.0]
1 (2.0, 22.0]
2 (22.0, 13102.0]


Now, in the process of making the graph, I encountered a lot of problems that I ended up working through piecemeal; they'll be explained in the markdown cell directly after the graph below.  But, after figuring out what I needed to do, I cleaned up this notebook, and just kept what worked.  So, for now, just accept that the next two cells are necessary.  First, I move the city names in the summary dataframe out of the index and into its own column, which gets named 'city'.  Then, I specifically write a new column containing the text that I'll want to show up as hovertext in the plot.

In [105]:
city_records.reset_index(inplace=True)
city_records.rename(columns={'index':'city'}, inplace=True)

In [106]:
city_records['text'] = np.nan
for i, row in city_records.iterrows():
    city_records.loc[i, 'text'] = '{}: {} ship(s)'.format(row['city'], row['containers'])

In [107]:
import plotly.plotly as py
from plotly.tools import set_credentials_file
import pandas as pd

set_credentials_file(username=plotly_username, api_key=plotly_api_key)

q = 3
city_records['quant'] = pd.qcut(
    x=city_records['containers'], q=q, precision=0, duplicates='drop')

colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","lightgrey"]
cities = []
scale = 3
log_scale = 100

for i, (interval, group) in enumerate(city_records.groupby('quant')):
    lim = interval
    city = dict(
        type = 'scattergeo',
        locationmode = 'ISO-3',
        lon = group['long'],
        lat = group['lat'],
        text = group['text'],
        marker = dict(
#             size = group['containers'] * scale, 
            size = np.log(group['containers']) * log_scale,
            color = colors[i],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '{:,} - {:,} ships sent'.format(
            np.rint(lim.left).astype(int),
            np.rint(lim.right).astype(int)) 
    )
    cities.append(city)

layout = dict(
        title = 'Ports of Lading for Ships <br />Arriving in Seattle in 2017',
        showlegend = True,
        geo = dict(
            scope='world',
            projection=dict( type='Mercator' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict( data=cities, layout=layout )
py.iplot( fig, validate=False, filename='sea2017_ports_of_lading' )

## Lessons Learned from First Plotly Experience

Ok, this one took me a long time to figure out.  It was kind of a chain reaction of failures in adapting the code from the [plotly bubble map example](https://plot.ly/python/bubble-maps/#united-states-bubble-map) where they varied the size of city markers based on population.  At first, I tried mostly just copying their entire set of code, and substituting my dataframe's lat and lon columns for theirs, but when it came to the legend, I didn't want to just shoehorn my data into their general approach.

The tutorial example used 5 "limit" categories, defining the quantiles of population ranks for US cities.  The thinking was obviously to make a dynamic legend that you could click on to reduce the clutter of the map to focus on specific subsets of cities: the 50th through 3000th largest cities in the US are far more numerous and far less populous than the top 50; you might want to turn off the small cities if all you care about is major metropolitan areas.  Conversely, NYC leaves an obnoxiously large imprint on the map, covering up other areas that you might want to be able to check out, so you can turn off NY and LA and see the rest of the map better, if that's your focus.

But their explicit assignment of those rank intervals was a kind of inflexible approach, so I wanted to redefine the legend to be broken up into cities that send more or less stuff to Seattle on the basis of the `df.qcut` method, which breaks your rows up into quantiles of some category, then pass those to the dict-building `for` loop from the tutorial code in order to build my own custom legend.  I tried initializing the `for` loop with:

    for interval, group in city_records.groupby('quant'):

That worked ok for passing lat and long coords straight from the dataframe, but when I went to adapt the hover text to display the number of ships that each city sent, I ran into trouble.  I tried using:

    text = '{}: {} ships'.format(group.index, group['containers'])
    
But whereas the `lat`, `lon`, and marker.size categories would accept arrays as input and parse them correctly, when I tried to set the text that way, it ended up passing the same array for every single row, breaking the hover text.

So then I got the bright idea to break out the data within the `groupby for` loop into individual rows, using:

    for interval, group in city_records.groupby('quant'):
        for index, row in group.iterrows():
        
And then set every lat, lon, and text value individually with each row's unique index.  And when I ran the cell, it would execute without complaint, but the resulting map would be empty: no legend, and no city markers.  I had previously noted that the cities might not be visible if you set the marker scale too small, in which case you have to fiddle with that parameter to make them apparent, but you could still always get hovertext when you moused over the map, meaning that the cities were still being plotted.  This time, the hovertext didn't show; plotly was obviously not successfully reading the cities specified by the `for` loop.

I tried outputting the `cities` list of dicts and parsing it by eye to see if it made sense, and compared the formatting to the list of dicts output by the tutorial example (of US cities, sized by population), and couldn't detect any missing fields or other dumb mistakes that would cause it to fail to plot.  I tried a lot of comparisons to no effect; the dict looked ok.  Then, finally, I happened to try checking the last dict in the tutorial example, and noticed that it looked very different from the first.  It turns out that the `for` loop from the tutorial doesn't write an individual dict for each city; it writes one dict for each legend item, or group of cities: one for each of the five groups of ranked city sizes that the author of the tutorial defined in their example.  So plotly wasn't expecting 77 dicts in the list `cities`, each with their own neatly-contained and well-defined keys and values, but rather a dict with an array of lat coords, an array of lon coords, and, for the value assigned to key 'marker', a dict containing an array of sizes corresponding to each city.

I tried wrenching the nested `for` loop using `iterrows` to fit this output, but couldn't get it to work.  So I abandoned the nested loop and went back to assigning variables from just the `group`s within `groupby`, but was still stuck with the problem of not being able to dynamically assign text.  So I reverted to the approach they took in the tutorial example: they explicitly create a new column containing the data from the other columns already in the dataframe.  Then you can pass an array of previously-defined strings to plotly in a way it apparently can understand, rather than making every entry the exact same array, with too much info for each city.

I had a little further trouble assigning the new `'text'` column dynamically with the iterrows function and a `'{}: {} ship(s)'.format()` call; it turns out you [can't initialize an entirely new column that way](https://stackoverflow.com/a/31460668/8637821), so you create a new column just with `np.nan` first, and use the for loop to replace them.  Finally, pandas couldn't find the city names as indices when you use iterrows in that way, and I noticed that the tutorial example just had default integer row indices, so I had to use `reset_index` to move the city names into their own column, rename the column so that it wouldn't get confused with the actual row index any more, and finally got the plot to render with the city name and number of shipments as hovertext (hence the two cells above the plot, resetting the index and specifying a new text column for each city, prior to setting up the plot itself).

~~The final wrinkle for now was that, even though you could now hide Vancouver by clicking on the proper legend dot, the other cities were still too small to be visible easily on the same scale, so I adapted the size variable by calling `np.log(group['containers'])` to reduce the variation between the sizes, then, since the markers were now too small, multiplied, rather than dividing, by the scale factor.  That's a bit of a fudge, but we all make compromises when visualizing data.~~  I go back and forth on this last point; I think that having their sizes manageable to see is useful, but taking the log really makes the range of shipping less intuitive.  You can comment out either of the above `size` parameters in the `markers` dict in order to see either approach.

Note, also, that you can dynamically alter the graph by changing the number of quantiles set by the first arg, '`q`'.  But the `qcut` method seeks to make all the bins of the same size, and since the most cities send very very few ships, you end up with multiple quantiles that contain just one or two records, and since the intervals must be unique, it can't display them separately, meaning you can't get all that many quantiles to display this way.  You can avoid getting warnings for overlapping quantiles by calling the arg `drop_duplicates=True`, but it'll still plot fewer than you call for.  There may be a way to skew the quantiles to preferentially break up the groups that contain more ships, but I haven't found a way to do this yet.

## Checking Against Other Cities/Years

Now, the task is to take what I've learned, and generalize the pipeline so I can easily feed in different time ranges or cities of destination in order to do comparisons with others from the Enigma bills of lading dataset.  I think a good workflow would be to have a few cells that could accept a csv file containing a subset of the Enigma db's shipping data from any given year, focused on the city of destination, and then reformat the city names, retrieve coordinates, and map them by number of containers shipped, as I've done above for Seattle in 2017.  It would be cool to have a more programmatic interface to cut out some of the human-centric downloading and moving files around, and their [API](http://docs.enigma.com/public/public_v20_user_pandas.html#api_export) should ultimately be compatible with this, but for now I'll settle with just having the work I did above to map one city's imports extrapolate to others in a matter of minutes.

Let's start with comparing Seattle's imports to that of a few other ports.  According to the frequency rank view of the summary statistics for the [Bill of Lading Summary db in Enigma for 2017](https://public.enigma.com/datasets/0293cd20-8580-4d30-b173-2ac27952b74b), the busiest ports that year were:

    Long Beach, California:   118,493 records
    Los Angeles, California:  105,329
    New York/Newark, NJ:       94,620
    Houston, Texas:            39,555
    Tacoma, Washington:        37,106
    
I had previously heard from a friend that Long Beach was the busiest US port for cargo traffic, but didn't know that LA proper has almost as much total activity; together, SoCal is definitely the hub for import in the US.  Coming in at a close third is no surprise: NYC has been the commercial hub for the country for a long time, although they moved the bulk of traffic from being processed in Manhattan [over to New Jersey in the mid-20th century](https://en.wikipedia.org/wiki/New_York_Harbor#Container_shipping_and_air_travel).  It's interesting that Houston is next; I hadn't heard of them being a particularly big depot, although perhaps it makes sense, as it's geographically closer to the center of the continental states than New York or LA.  Finally, I had also previously heard that Tacoma is larger than Seattle when it comes to cargo traffic.

That's a good cross-section of major commercial traffic in the US, let's try plotting them and seeing whether the city of origin for imports varies based on their destination among US ports.  I'll import the data, reformat the city names, download coords, and try to get them to plot in different subplots with plotly.

In [25]:
# Specify a list of files to be opened, processed, and plotted
cities_2017_paths = {'Long_Beach': paths['longbeach_shipping_2017_csv'],
                     'Los_Angeles': paths['losangeles_shipping_2017_csv'],
                     'NYC_Newark': paths['nyc_shipping_2017_csv'],
                     'Houston': paths['houston_shipping_2017_csv'],
                     'Tacoma': paths['tacoma_shipping_2017_csv'],
                     'Seattle': paths['seattle_shipping_2017_csv']}
cities_2017_dfs = {}

# Import each into Pandas
for city, path in cities_2017_paths.items():
    cities_2017_dfs[city] = pd.read_csv(path)
    # Add another level to column names to keep grouped together by city
#     cities_2017_dfs[city].columns = \
#     pd.MultiIndex.from_product([cities_2017_dfs[city].columns, [city]])
    # Combine into one df
#     cities_2017_df = pd.merge(cities_2017_dfs.values(), how='outer')

In [25]:
# Check for ports of lading with numeric code, rather than names
def code_lookup(dict_of_dfs):
    matches_set = set()
    for city, df in dict_of_dfs.items():
        matches = df['foreign_port_of_lading'].str.match(r'[\d]+')
        matches_series = df.loc[matches, 
                                'foreign_port_of_lading'].value_counts()
        matches_set.update(matches_series.index.values)
        print('{}:\n{}'.format(city, matches_series))
    return matches_set

code_matches = code_lookup(cities_2017_dfs)

Houston:
Series([], Name: foreign_port_of_lading, dtype: int64)
Long_Beach:
Series([], Name: foreign_port_of_lading, dtype: int64)
Los_Angeles:
Series([], Name: foreign_port_of_lading, dtype: int64)
NYC_Newark:
Series([], Name: foreign_port_of_lading, dtype: int64)
Seattle:
Series([], Name: foreign_port_of_lading, dtype: int64)
Tacoma:
Series([], Name: foreign_port_of_lading, dtype: int64)


### Automated City Lookup?

An aside here: I the list of numerically-coded ports of lading above is manageable, and I could easily create a dict to replace them with the appropriate city, just by reading through [this document](https://www.cbp.gov/sites/default/files/assets/documents/2017-Feb/appendix_f_0.pdf).  But to ensure that future, unanticipated codes could be dealt with automatically, I wondered if it would be possible to just parse the PDF that contains those codes, and automatically generate code: city pairs.

I searched for modules for extracting text from PDFs, and heard to try `textract`.  When pip installing it, however, it returns an error because one of its dependencies, `EBookLib`, has a readme file within its pip wheel that contains non-standard characters, that cause it to throw an error.  I found a [thread on this](https://github.com/deanmalmgren/textract/issues/194), and it sounded do-able.  I opened the offending file in Notepad++, replaced the string, and saved, but after clearing the pip cache and re-running, I hit the same error.  The helpful author had [another suggestion](https://github.com/deanmalmgren/textract/issues/194#issuecomment-357416217) about editing the file in pip, but while that code executed, the same error results when trying `pip install textract`.  I verified that the README file had been successfully modified to remove the characters in question, but it just never worked.

I finally got the pip install to work by just skipping to the end of the thread and running [this command](https://github.com/deanmalmgren/textract/issues/194#issuecomment-378054308).  I probably spent a couple of hours just trying to get this stupid module.  I'll try it below and see if it's suitable for automating the code-name replacement.

In [16]:
import textract
import requests

url = 'https://www.cbp.gov/sites/default/files/assets/\
documents/2017-Feb/appendix_f_0.pdf'
paths['codes_pdf'] = os.path.join(paths['data_dir'], 'codes.pdf')

response = requests.get(url)
with open(paths['codes_pdf'], 'wb') as f:
    f.write(response.content)

pdf_file = open(paths['codes_pdf'], 'rb')
pdf_data = textract.process(paths['codes_pdf'])

ModuleNotFoundError: No module named 'textract'

In [ ]:
pdf_data[:1000]

Ok, that's not as clean as I would like, but a pattern is visible.  The country names are preceded by '\r\n\r\n', then the name ('Albania'), then there's just '\r\n' between all of the codes, then there's another '\r\n\r\n' between the last code and the first city name, then there's another '\r\n\r\n' before the next country name ('Algeria'), and the pattern repeats.  In other words, there is [one ASCII carriage return and one 'linefeed'](https://docs.python.org/2.0/ref/strings.html) between each country name and the first code, between each adjacent pair of codes, and between each city name, but there are two of each of those breaks between the last city name and the next country, as well as between the last code and the first city name.  That isn't **too** complicated, but I was hoping that the text would be interpreted as a more direct spatial coordination between code and city name.

I definitely could write a custom function that would parse this, but I've already spent enough time on this.  For the purposes of this notebook, I'll just create a dict manually to handle the missing city names from the dataframes above.  Unfortunately, many of the codes couldn't be found, either on the PDF listed above or a couple other references.  The codes are consistent enough to tell what country they're referring to, but the cities are missing, and I choose not to guess by picking the closest integer number.

I won't mess around with more annotation here; I'll just write one function to parse the dfs up to the point of obtaining their coords, then save the results as pickle files.

## Consolidating Data Aggregation/Cleanup; Applying to Other Ports of Unlading

I'm going to try to recapitulate the process I used above to get the lat-long coords for ports of lading to Seattle, and extend it as smoothly as possible to the other cities.  I'll see how succinctly I can achieve that.

In [52]:
# Specify a list of files to be opened, processed, and plotted
cities_2017_paths = {'Long_Beach': paths['longbeach_shipping_2017_csv'],
                     'Los_Angeles': paths['losangeles_shipping_2017_csv'],
                     'NYC_Newark': paths['nyc_shipping_2017_csv'],
                     'Houston': paths['houston_shipping_2017_csv'],
                     'Tacoma': paths['tacoma_shipping_2017_csv'],
                     'Seattle': paths['seattle_shipping_2017_csv']}

In [53]:
import json

# DCM code for retrieving credentials
paths['plotly_cred'] = os.path.join(paths['data_dir'], 'plotly_api_credentials.json')
paths['google_cred'] = os.path.join(paths['data_dir'], 'google_api_cred.json')

with open(paths['plotly_cred'], 'r') as f:
    plotly_cred = json.load(f)
with open(paths['google_cred'], 'r') as f:
    google_cred = json.load(f)
    
plotly_username = plotly_cred['username']
plotly_api_key = plotly_cred['api_key']
google_api_key = google_cred['api_key']

In [154]:
import re
from geopy import geocoders

# Specify the dict of codes to replace with city names
origin_codes_dict = {'57073': 'Nansha, China (Mainland)', 
                     '53304': 'Pipavav, India', 
                     '30110': 'Covenas, Columbia', 
                     '57037': 'Yangshan, China (Mainland)'
                    }


def import_shipping_data(paths):
    cities_dfs = {}
    # Import each port of unlading's shipping records into Pandas
    for city, path in paths.items():
        cities_dfs[city] = pd.read_csv(path)
    return cities_dfs


def replace_city_codes(cities_dfs, codes_dict):
    # Initialize counters to print how many records were dropped
    records_dropped = 0
    total_records = 0
    replaced = {}
    # First, see if any 5-digit codes present in dfs
    # were missing from dict; if so, add to dict as NaN
    for city, df in cities_dfs.items():
        matches = df.loc[:,'foreign_port_of_lading'].str.match(r'[\d.]{5}')
        match_list = df.loc[matches, 
                            'foreign_port_of_lading'].unique().tolist()
        for match in match_list:
            if match not in codes_dict:
                codes_dict[match] = np.nan
                print('"{}" not found in input dict; \
adding to dict as np.nan.'.format(match))
            else:
                pass
        # Replace codes with cities where possible; drop the rest
        new_df = df.copy()
        new_df.loc[:,'foreign_port_of_lading'].replace(codes_dict, 
                                                       inplace=True)
        # No point keeping record with no city listed
        records_dropped += new_df['foreign_port_of_lading'].isnull().sum()
        total_records += new_df.shape[0]
        new_df = new_df[new_df.loc[:,'foreign_port_of_lading'].notnull()]
        replaced[city] = new_df
    # Add a print line to summarize how many records were discarded
    print('\n{} records were dropped from bills of lading \
because no foreign port was found with the numeric code given; \
amounting to {:.8f}% of all records.'.format(
    records_dropped, records_dropped*100/total_records))
    return replaced


def get_words(source):
    words = re.compile('[A-z]+')
    # Get rid of '(Mainland)' and other parenthetical stuff
    noparens = re.sub(r'\([^)]*\)', '', source)
    # Separate city from country
    parts = noparens.split(sep=',')
    formatted = []
    # Process city and country separately
    for part in parts:
        new_parts = []
        # Get rid of leading/trailing whitespace
        part = part.strip()
        # Discard punctuation, numbers; any non-words
        parts_words = re.findall(words, part)
        # Separate individual words by a single space
        new_parts.append(' '.join(parts_words))
        # Re-join city and country, separated by comma and space
        formatted.append(', '.join(new_parts))
    return str(', '.join(formatted))


def format_ports(cities_dfs):
    new_port_dfs = {}
    # Use pandas' vectorized function, 
    # rather than iterating over rows slowly
    for city, df in cities_dfs.items():
        df.loc[:,'foreign_port_of_lading'] = df.loc[:,
            'foreign_port_of_lading'].astype(str).apply(get_words)
        new_port_dfs[city] = df
    return new_port_dfs


def geolocate(cities_dfs):
    # ArcGIS no longer tolerates me; use Google
    geolocator = geocoders.GoogleV3(api_key=google_api_key,
                                    timeout=100)
    all_foreign = set()
    lat_long = dict()
    failures = []
    # Don't spam the server with redundant calls;
    # first gather all unique cities together
    for city, df in cities_dfs.items():
        all_foreign.update(
            df.loc[:,'foreign_port_of_lading'].unique().astype(str))
    # Feed all unique cities to server, save output
    # Either in dict with coords, or else as failures
    for port in all_foreign:
        location = geolocator.geocode(port)
        if location:
            lat = location.latitude
            long = location.longitude
            lat_long[port] = (lat, long)
        else:
            failures.append(port)
    return lat_long, failures

            
def lookup(port, lat_long_dict):
    if port in lat_long_dict.keys():
        coords = lat_long_dict[port]
    else:
        coords = np.nan
    return coords


def get_coords(cities_dfs, lat_long_dict):
    coordinated = {}
    # Again, use vectorized functions
    for city, df in cities_dfs.items():
        new_df = df.copy()
        new_df['lat'] = np.nan
        new_df['long'] = np.nan
        new_df['lat_long'] = np.nan
        new_df['lat_long'] = new_df['foreign_port_of_lading'].astype(
            str).apply(lookup, lat_long_dict=lat_long_dict)
        lat_dict = {}
        long_dict = {}
        for port, coords in lat_long_dict.items():
            lat_dict[port] = coords[0]
            long_dict[port] = coords[1]
        new_df['lat'] = new_df['foreign_port_of_lading'].astype(
            str).apply(lookup, lat_long_dict=lat_dict)
        new_df['long'] = new_df['foreign_port_of_lading'].astype(
            str).apply(lookup, lat_long_dict=long_dict)
        # drop any without coords
        empties = new_df.loc[:,'lat_long'].isnull().sum()
        print('{:<12}\nCoordinates couldn\'t be found for {:>5} \
ports of lading; {:>.2f}% of all records'.format(
            city, empties, empties/new_df.shape[0]*100))
        new_df.dropna(subset=['lat_long'], inplace=True)
        coordinated[city] = new_df
    return coordinated


It took me significantly longer than I expected to refactor the approach that worked above for Seattle into a series of functions that would make for a straightforward pipeline to modify the dataframes for several ports of unlading.  I still don't have a lot of experience writing extended pipelines of functions; I could've left a lot of the code the same, but wanted to be able to invoke the whole process in as few lines as possible.

I ended up having to write two new functions (`format_ports` and `get_coords`) to replace the `.apply(custom_func)` calls that I used in the original case to both reformat the `foreign_port_of_lading` strings and then to populate the `lat_long` column from the dict.  I could've left those calls separate but, again, I wanted to simplify.  I could also still write a single wrapper function to reduce the 5 lines in the cell below with a single call, but I found it useful to keep them separate while troubleshooting the various functions.  Now that they work I'll still keep them separate.

I'm aware that defining a single class, with each of these as methods within, is probably the canonical, Pythonic way to go, but I have even less experience at present with proper class coding, so I'll pass on that for now.

In [66]:
%%time

cities_2017_dfs = import_shipping_data(cities_2017_paths)
cities_2017_dfs = replace_city_codes(cities_2017_dfs, origin_codes_dict)
cities_2017_dfs = format_ports(cities_2017_dfs)
# lat_long, failures = geolocate(cities_2017_dfs)
cities_2017_dfs = get_coords(cities_2017_dfs, lat_long)

Long_Beach  
Coordinates couldn't be found for  1762 ports of lading; 1.49% of all records
Los_Angeles 
Coordinates couldn't be found for  1312 ports of lading; 1.25% of all records
NYC_Newark  
Coordinates couldn't be found for    71 ports of lading; 0.08% of all records
Houston     
Coordinates couldn't be found for    54 ports of lading; 0.14% of all records
Tacoma      
Coordinates couldn't be found for  3186 ports of lading; 8.59% of all records
Seattle     
Coordinates couldn't be found for     1 ports of lading; 0.00% of all records
Wall time: 1.86 s


In [67]:
cities_2017_dfs.keys()

dict_keys(['Long_Beach', 'Los_Angeles', 'NYC_Newark', 'Houston', 'Tacoma', 'Seattle'])

In [68]:
cities_2017_dfs['Seattle'].head()

,identifier,trade_update_date,run_date,vessel_name,port_of_unlading,estimated_arrival_date,foreign_port_of_lading,record_status_indicator,place_of_receipt,port_of_destination,...,description_sequence_number,piece_count,description_text,harmonized_number,harmonized_value,harmonized_weight,harmonized_weight_unit,lat,long,lat_long
0,201712261102,2017-11-22T00:00:00,2017-12-26T00:00:00,NEW YORK EXPRESS,"Seattle, Washington",2017-12-21T00:00:00,"Hong Kong, Hong Kong",New,NANSHA CHINA,NaN,...,1,127,"SOFA & UPHOLSTERY PO418287/418285/418967 ""SH """,NaN,NaN,NaN,NaN,22.396428,114.109497,"(22.396428, 114.109497)"
1,201712261102,2017-11-22T00:00:00,2017-12-26T00:00:00,NEW YORK EXPRESS,"Seattle, Washington",2017-12-21T00:00:00,"Hong Kong, Hong Kong",New,NANSHA CHINA,NaN,...,1,136,"SOFA & UPHOLSTERY PO418287/418285/418967 ""SH """,NaN,NaN,NaN,NaN,22.396428,114.109497,"(22.396428, 114.109497)"
2,201712261102,2017-11-22T00:00:00,2017-12-26T00:00:00,NEW YORK EXPRESS,"Seattle, Washington",2017-12-21T00:00:00,"Hong Kong, Hong Kong",New,NANSHA CHINA,NaN,...,1,56,"SOFA & UPHOLSTERY PO418287/418285/418967 ""SH """,NaN,NaN,NaN,NaN,22.396428,114.109497,"(22.396428, 114.109497)"
3,201712261237,2017-11-23T00:00:00,2017-12-26T00:00:00,YM UNANIMITY,"Seattle, Washington",2017-12-20T00:00:00,"Tsingtao, China",New,QINGDAO,NaN,...,1,1263,PE TARPAULIN THIS SHIPMENT CONTAINS NO WOOD P ...,NaN,NaN,NaN,NaN,36.067108,120.382609,"(36.067108, 120.382609)"
4,201712261259,2017-11-29T00:00:00,2017-12-26T00:00:00,NEW YORK EXPRESS,"Seattle, Washington",2017-12-21T00:00:00,"Pusan, South Korea",New,DALIAN CHINA,NaN,...,1,37,WOODEN DOORS,NaN,NaN,NaN,NaN,35.179554,129.075642,"(35.1795543, 129.0756416)"


Ok, not too long a time to wait, considering the amount of data being processed.  Most of it was in the lookup time for retrieving the coordinates from Google using `geolocate`.  Note that, while I used ArcGIS for the Seattle df earlier in this notebook, it repeatedly refused to process so many ports of origin this time around.  It [sounds like](https://developers.arcgis.com/rest/geocode/api-reference/geocoding-free-vs-paid.htm) they aren't expecting you to do bulk access of their geolocator db, maybe just a few records at a time.  So I switched to [GoogleV3](https://developers.google.com/maps/documentation/geocoding/start), got an API key, and they were more patient with me (at least once I changed the default `timeout=1` arg upon initialization).  Let's see how many coords are missing:

In [95]:
for city, df in cities_2017_dfs.items():
    if df['lat_long'].isnull().sum() != 0:
        print('{:<15}{:.6f}'.format(
            city, df['lat_long'].isnull().sum()/df.shape[0]))

In [96]:
for i, failure in enumerate(failures):
    print('{:<4}{}'.format(i, failure))

0   North Pacific, United States Off Shore Tanker Transshipment Points
1   High Seas, United States Off Shore Tanker Transshipment Points
2   All Other Egypt Mediterranean Region Ports, Egypt
3   Hitaki Ko, Japan
4   All Other Netherlands Antilles Ports, Netherland Antilles
5   Khor al Ami, Kuwait
6   Nagoya Ko, Japan
7   Puduchcheri, India
8   Gulf of Mexico, United States Off Shore Tanker Transshipment Points
9   Port Redon, Vietnam
10  All Other Estonia Ports, Estonia
11  Soyo Quinfuquena term, Angola


So there were significantly more records lost for destinations other than Seattle because Google didn't find the coordinates of the port of lading.  But from looking at the list of the `failures`, I can't blame Google at all; those are some strange, wonky-sounding origins in several cases.  The phenomenon of "transshipment" [throws a considerable monkey wrench into things](https://www.usitc.gov/publications/332/ec200404b.pdf), and I'm not gonna solve that issue here.

Okay!  Now I'm ready to plot the various ports of lading for these respective US ports of unlading.  I'll drop the rows lacking coord data; I don't expect attempting to cover the exceptions found in `failures` will prove very edifying, and they still collectively represent around 1% or less for each of the cities (except Tacoma; and it could be interesting to look into why someday).

In [149]:
cities_2017_dfs_clean = {}
for city, df in cities_2017_dfs.items():
    print('{:<12} size before dropna: \t{}'.format(city, df.shape[0]))
    cities_2017_dfs_clean[city] = df[df['lat_long'].notnull()]
    print('{:<12} size after dropna: \t{}'.format(
        city, cities_2017_dfs_clean[city].shape[0]))

Long_Beach   size before dropna: 	118483
Long_Beach   size after dropna: 	116721
Los_Angeles  size before dropna: 	105320
Los_Angeles  size after dropna: 	104008
NYC_Newark   size before dropna: 	94219
NYC_Newark   size after dropna: 	94148
Houston      size before dropna: 	39071
Houston      size after dropna: 	39017
Tacoma       size before dropna: 	37106
Tacoma       size after dropna: 	33920
Seattle      size before dropna: 	27099
Seattle      size after dropna: 	27098


Ok, now I'll just want to pickle these (as well as the lat_long dict), so I can access them later:

In [150]:
for city in cities_2017_dfs.keys():
    name = '{}_2017_pkl'.format(city)
    filename = '{}_2017.pkl'.format(city)
    paths[name] = os.path.join(paths['data_dir'], filename)
    
for city, df in cities_2017_dfs_clean.items():
    name = '{}_2017_pkl'.format(city)
    with open(paths[name], 'wb') as f:
        pickle.dump(df, f)
        
paths['lat_long_dict'] = os.path.join(paths['data_dir'], 
                                      'lat_long_dict.pkl')
with open(paths['lat_long_dict'], 'wb') as f:
    pickle.dump(lat_long, f)
    
paths['unlading_coords'] = os.path.join(paths['data_dir'], 'top_6_unlading.pkl')

with open(paths['unlading_coords'], 'wb') as f:
    pickle.dump(unlading_coords, f)

# If Resuming Notebook, Start With Cell Below

In [68]:
# I often switch between editing notebooks from my PC to my Mac;
# the code in this cell is to find the proper local data when I do so.

import platform
import os
import locale
import numpy as np
import pandas as pd
import multiprocessing as mp
import pickle
import json

o_s = platform.system()
paths = {}

if o_s == 'Darwin':
    paths['data_dir'] = '/Users/drew/data/Enigma'
elif o_s == 'Windows':
    paths['data_dir'] = r'C:\Users\DMacKellar\Documents\Data\
\Enigma'

locale.setlocale(locale.LC_ALL, '')

# DCM code for retrieving credentials
paths['plotly_cred'] = os.path.join(paths['data_dir'], 'plotly_api_credentials.json')
paths['google_cred'] = os.path.join(paths['data_dir'], 'google_api_cred.json')

with open(paths['plotly_cred'], 'r') as f:
    plotly_cred = json.load(f)
with open(paths['google_cred'], 'r') as f:
    google_cred = json.load(f)
    
plotly_username = plotly_cred['username']
plotly_api_key = plotly_cred['api_key']
google_api_key = google_cred['api_key']

for root, dirs, files in os.walk(paths['data_dir']):
    for file in files:
        file_under = str(file).replace('.', '_')
        path = os.path.join(root, file)
        paths[file_under] = path
        
# That should populate all data files.  Whenever a loading call fails, do:
# print(*paths.keys(), sep='\n')
# And look for the key with the proper name (with underscore instead of period)

# For now, just load the 6 dfs with lat_long coords
cities_2017_dfs = {}
for path in paths:
    if path[-9:] == '_2017_pkl':
        with open(paths[path], 'rb') as f:
            name = str(path[:-9])
            cities_2017_dfs[name] = pickle.load(f)

# Load api credentials
with open(paths['credentials_json'], 'r') as f:
    credentials = json.load(f)

In [69]:
# For each port of unlading, aggregate ports of lading
def get_counts(dict_of_dfs):
    city_counts_dfs = {}

    for city, df in dict_of_dfs.items():
        # Getting tired of typing out 'foreign_port_of_lading' all the time
        df1 = df.copy()
        df1.rename(columns={'foreign_port_of_lading':'city'}, inplace=True)
        grouped = df1.groupby(df['city'])
        containers = grouped.size().to_dict()
        df1['containers'] = df1.loc[:, 'city'
                                 ].apply(lambda x: containers[x])
        cols = ['city', 'lat_long', 
                'lat', 'long', 'containers']
        unique = df1.drop_duplicates(subset=['city'])
        city_records = unique.loc[:, cols]

        # Make a column to contain the 'text' data you want in hovertext:
        city_records['text'] = np.nan
        for i, row in city_records.iterrows():
            city_records.loc[i, 'text'] = '{}: {} container(s)'.format(
                row['city'], row['containers'])
        city_counts_dfs[city] = city_records
    return city_counts_dfs

In [70]:
city_counts_dfs = get_counts(cities_2017_dfs)

In [71]:
city_counts_dfs['Seattle'].head()

,city,lat_long,lat,long,containers,text
0,"Hong Kong, Hong Kong","(22.396428, 114.109497)",22.396428,114.109497,940,"Hong Kong, Hong Kong: 940 container(s)"
3,"Tsingtao, China","(36.067108, 120.382609)",36.067108,120.382609,23,"Tsingtao, China: 23 container(s)"
4,"Pusan, South Korea","(35.1795543, 129.0756416)",35.179554,129.075642,3240,"Pusan, South Korea: 3240 container(s)"
5,"Yantian, China","(22.556499, 114.236875)",22.556499,114.236875,999,"Yantian, China: 999 container(s)"
11,"Shanghai, China","(31.2303904, 121.4737021)",31.230390,121.473702,2936,"Shanghai, China: 2936 container(s)"


It looks like [this](https://plot.ly/python/map-subplots-and-small-multiples/) is the right reference for how to make multiple subplots with maps in plotly.  That example, however, does not use hovertext, and furthermore, each subplot only has one trace.  In the previous example from this notebook, where I made one map for Seattle, I grouped the ports of lading into separate traces, and had the city name and ships sent as the mouse-over text.  It'll require [some modification](https://community.plot.ly/t/adding-traces-to-subplots/4424) to the hyperlinked example code to get this to work.  As a fall-back, maybe I can abandon having multiple traces of groupings of ports of lading, and just use log scaling to make all markers visible on each subplot (since I wouldn't then be able to turn each group off individually).

Ok, after playing around with the simpler, `fig.append_trace()` functionality, it's returning errors that "`'xaxis' is not allowed in 'scattergeo'`".  It looks like the normal way to do this is to specify a unique '`geo`' arg for each dict in `data`, then add a dict with that '`geo`' label as a keyword within the dict `layout`, and the value for that 'geo' keyword will itself be a dict, which will contain the keyword 'domain', the value of which will be another dict, which will have two keys, 'x' and 'y', each of which will have a list/array with two elements, which will be the limits along the entire figure's x and y axis, as to the outer bounds for that subplot.  [This example](https://plot.ly/python/mixed-subplots/) may be more fruitful a source of inspiration.

So, the syntax appears to be:
    
    trace3 = {
        "geo": "geo3", 
        "lon": df['Longitude'],
        "lat": df['Latitude'],
        "hoverinfo": 'text',
        "marker": {
            "size": 4,
            "opacity": 0.8,
            "color": '#CF1020',
            "colorscale": 'Viridis'
        }, 
        "mode": "markers", 
        "type": "scattergeo"
    }
    trace2 = {
        "geo": "geo4"
        ...
    }
    
    data = [{trace1}, {trace2}]
    
    layout = {
        ...
        "geo3": {
            "domain": {
                "x": [0, 0.55], 
                "y": [0, 0.9]
        }, 
        
    }
    
It appears that, to get multiple traces in one subplot, you just use the same 'geo' keyword for each, dict appended to data; the layout will then group them all together.  So that's the reason for the somewhat more convoluted way the first example cited above (the one that outputs multiple US maps with WalMart stores over the years): plotly doesn't let you pass a unique axis object associated with each `data` dict when the object is a map, it instead requires that you specify what part of the [0, 1] space of the entire figure's x and y axes you want each trace to occupy, and you indicate that in the layout dict, not within the dict sent to the `data` list for each trace.

In [32]:
import plotly.plotly as py
import plotly.tools as tools
import pandas as pd
import itertools

def plotly_subplots(dict_of_dfs, title, q=4, scale=1, rows=3, cols=2):
    tools.set_credentials_file(username=plotly_username, 
                               api_key=plotly_api_key)

    q = 4
    colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)",
              "rgb(255,133,27)","lightgrey"]
    data = []
    scale = scale

    layout = dict(
        title = title,
        font=dict(family='Arial, sans serif',
                  size=22,
                  color='rgb(0, 0, 0)'
                ),
        # showlegend = False,
        autosize = True,
        width = 1000,
        height = 900,
        hovermode = True,
        legend = dict(
            x=1,
            y=0.5,
            bgcolor="rgba(255, 255, 255, 0)",
            font = dict( size=11 ),
        )
    )

    # Want one subplot per port of unlading, but multiple groups
    # of ports of lading per subplot
    geo_s = []
    # Iterate over dict of dfs
    for i, (city, df) in enumerate(dict_of_dfs.items()):
        # Split each df up into quantiles of ports of lading
        # Need a special case for Seattle, to keep # groups right
        if city == 'Seattle':
            q = 5
        else:
            q = 4
        df['quant'] = pd.qcut(
            x=df['containers'], q=q, precision=0, duplicates='drop')
        # Specify geo_key, which will tell 
        # layout_dict where to place subplot
        geo_key = 'geo'+str(i+1) if i != 0 else 'geo'
        # Go through groups of cities and plot markers
        for j, (interval, group) in enumerate(df.groupby('quant')):
            lim = interval
            label = '{}: <br />{} - {} containers'.format(
                city.replace('_', ' '), 
                np.rint(lim.left).astype(int),
                np.rint(lim.right).astype(int)-1)
            group_data = dict(
                type = 'scattergeo',
                geo = geo_key,
                locationmode = 'ISO-3',
                lat = group['lat'],
                lon = group['long'],
                text = group['text'],
                marker = dict(
                    size = group['containers']/scale,
    #                 size = np.log(group['containers'])*50,
                    color = colors[j],
                    line = dict(width=0.5, color='rgb(40,40,40)'),
                    sizemode = 'area'
                ),
                name = label,
                legendgroup = label
            )
            data.append(group_data)

        # Add a clear map to show plot titles
        data.append(
            dict(
                type = 'scattergeo',
                showlegend = False,
                # Always put the text in the same place;
                #
                lon = [-115],
                lat = [-60],
                geo = geo_key,
                text = '{}'.format(city.replace('_', ' ')),
                mode = 'text',
                textfont=dict(
                    family='Arial, sans serif',
                    size=22,
                    color='rgb(0, 0, 0)'
                )
            )
        )

        layout[geo_key] = dict(
            scope='world',
            projection=dict( type='Mercator' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)",
            domain = dict( x = [], y = [] ),
        )


    z = 0
    COLS = cols
    ROWS = rows
    for y in reversed(range(ROWS)):
        for x in range(COLS):
            geo_key = 'geo'+str(z+1) if z != 0 else 'geo'
            layout[geo_key]['domain']['x'] = [float(x)/float(COLS), 
                                              float(x+1)/float(COLS)]
            layout[geo_key]['domain']['y'] = [float(y)/float(ROWS), 
                                              float(y+1)/float(ROWS)]
            z=z+1
            if z > len(data):
                break
    subp_data = data
    subp_layout = layout
    
    return data, layout

In [28]:
title = '<b>Ports of Lading Presented by <br />\
Ports of Unlading: 2017</b>'

data, layout = plotly_subplots(city_counts_dfs, title=title)

fig = { 'data':data, 'layout':layout }
py.iplot( fig, filename='Ports of Lading', height=900, width=1000 )

## My First Plotly Maps in Subplots Figure

Ok, after a lot of fiddling, I finally got that to look pretty much the way that I want.  With such a specific plotting agenda, it's probably difficult to avoid a lot of fiddling to get the parameters right.

Another word on the learning curve: plotly is based on D3.js, i.e. Javascript, and thus has a different kind of feel than the typical Python library, in my experience.  In particular, it seems to involve a lot more dicts, and nested dicts.  The easy access to maps is pretty cool, but for some reason it makes it incompatible with easy creation/access to subplots, thus the comment in the cell above about having to create text annotation via a bunch of independent traces mapped to the same figure-grid coordinates.  Walking through [the example code](https://plot.ly/python/map-subplots-and-small-multiples/) about Walmart stores was very helpful in getting the plot to eventually render for me.  Basically, you feed everything in to one data object (a list of dicts) and one layout object (a dict), and link the two with the `geo` key arg.

---

So, finally: time to consider some implications of the data.  Yes, Seattle is unique in receiving so much traffic from Vancouver.  Tacoma has far, far less, and in general processes more East Asian traffic, and less from Latin America or Europe.  LA and LBC process the most cargo, overall, and most of it from China: Hong Kong (Yantian) is tops for LA, and Shanghai sends the most to Long Beach.  Both also get a considerable amount from Latin America and Europe; kind of like Seattle's profile, in that regard.

**One significant point that I just realized: I've been saying (or at least implying) that individual rows in the data represent unique ships, but now see that they're individual containers, many of them on the same ship.  It's an example of not looking over your data enough in preliminary analysis before jumping into visualization and in-depth analysis.  So I've changed the legend above and will try to fix the rest of the notebook; if I overlook any prior to this point, please disregard.  Such a major oversight should have been fairly obvious, but I'm not really that familiar with the subject matter: 27 million containers a year is a lot more believable than 27 million ships in a year, but either one is still kind of hard to imagine.**

The two major ports that aren't on the West Coast are interesting: Houston gets a lot more traffic from the Gulf side of Mexico, and from the Atlantic coast of South America, as well as being a pretty major import hub from Europe.  Newark, however, is of course *the* entrepot of Atlantic traffic, although they do also get a lot from South Asia and even East Asia.

Some anomalies are interesting, however.  NYC receives a lot of cargo from a place called South Riding Point, Bahamas.  2,166 containers in 2017, to be exact.  There's no way that the Bahamas are a major exporter of just about anything on that scale, so this is probably more evidence of some kind of economic shenanigans wherein the place where goods are loaded for export isn't necessarily the same country where they're produced.  Other points: Africa and the Middle East aren't huge hotspots for exporting to the major US hubs, and, perhaps more surprisingly, neither is Australia.  This could be evidence that extremes of development in nations decrease the likelihood that it will be a big exporter to the US, but the amount of goods we get from Europe would seem to weaken such a conclusion.

Then again, this is just one look at some of the biggest ports in the US, and visual analysis with a map, while intuitive and immediate, is less thorough, and the largest markers easily dominate the visual field when doing a quick survey.  The 6 cities shown above account for just over 400,000 rows of shipping records from 2017, out of a total 27 million rows in the entire dataset.  It would be interesting to create a heatmap of all lat-long origins of shipping bound to the US, either from the entire dataset (if my laptop can handle it), or just from these cities.

## Average Lat-Lon for Ports of Lading

First, though, let's just try something extremely simple: just find the 'center of mass' for each port of unlading:

In [4]:
# # Note: Already run; load from paths['top_6_unlading_pkl']

# from geopy import geocoders

# unlading_ports = ['Long Beach, California', 'Los Angeles, California', 'Tacoma, Washington',
#                   'Seattle, Washington', 'Houston, Texas', 'Newark, New Jersey']
# unlading_coords = dict()
# failures = []
# geolocator = geocoders.GoogleV3(api_key=google_api_key,
#                                     timeout=100)
# for port in unlading_ports:
#         location = geolocator.geocode(port)
#         if location:
#             lat = location.latitude
#             long = location.longitude
#             unlading_coords[port] = (lat, long)
#         else:
#             failures.append(port)

with open(paths['top_6_unlading_pkl'], 'rb') as f:
    unlading_coords = pickle.load(f)
    
name_translator = dict()
for (key, city) in zip(sorted(unlading_coords.keys()), sorted(cities_2017_dfs.keys())):
    name_translator[city] = key
print(unlading_coords, '\n\n', name_translator)

{'Long Beach, California': (33.7700504, -118.1937395), 'Los Angeles, California': (34.0522342, -118.2436849), 'Tacoma, Washington': (47.2528768, -122.4442906), 'Seattle, Washington': (47.6062095, -122.3320708), 'Houston, Texas': (29.7604267, -95.3698028), 'Newark, New Jersey': (40.735657, -74.1723667)} 

 {'Houston': 'Houston, Texas', 'Long_Beach': 'Long Beach, California', 'Los_Angeles': 'Los Angeles, California', 'NYC_Newark': 'Newark, New Jersey', 'Seattle': 'Seattle, Washington', 'Tacoma': 'Tacoma, Washington'}


In [22]:
import plotly.plotly as py
import plotly.tools as tools
import plotly.graph_objs as go
import pandas as pd
import itertools

def plotly_avg_coords(dict_of_dfs, title, scale=1E3, width=1000, height=600):
    tools.set_credentials_file(username=plotly_username, 
                               api_key=plotly_api_key)

    colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)",
              "rgb(255,133,27)","rgb(40,40,40)",'rgb(100,200,50)']
    data = []
    scale = scale
    log_scale = 3

    layout = dict(
        title = title,
        font=dict(family='Arial, sans serif',
                  size=18,
                  color='rgb(0, 0, 0)'
                ),
        showlegend = True,
        autosize = False,
        width = width,
        height = height,
        hovermode = True,
        legend = dict(
            x=1,
            y=0.5,
            bgcolor="rgba(255, 255, 255, 0)",
            font = dict( size=11 ),
        ),
        geo = dict(
                scope='world',
                projection=dict( type='Mercator' ),
                showland = True,
                landcolor = 'rgb(217, 217, 217)',
                subunitwidth=1,
                countrywidth=1,
                subunitcolor="rgb(255, 255, 255)",
                countrycolor="rgb(255, 255, 255)"
            )
    )

    # Iterate over dict of dfs
    for i, (city, df) in enumerate(dict_of_dfs.items()):
        city_read = '{}'.format(city.replace('_', ' '))
        total_weights = df['containers'].sum()
        lat_weight = df['lat'] * df['containers']
        lat_avg = lat_weight.sum()/total_weights
        lon_weight = df['long'] * df['containers']
        lon_avg = lon_weight.sum()/total_weights
        un_lat = unlading_coords[name_translator[city]][0]
        un_lon = unlading_coords[name_translator[city]][1]
        geo_key = 'geo'

        # Go through cities and plot markers to 'center of mass'    
        center_of_mass = dict(
            type = 'scattergeo',
            geo = geo_key,
            locationmode = 'world',
            lat = [lat_avg],
            lon = [lon_avg],
            text = '"Center of mass" of ports <br />\
    exporting to {}'.format(city_read),
            showlegend = False,
            marker = dict(
                size = df['containers'].sum()/scale,
    #             size = np.log(df['containers'].sum())*log_scale,
                color = colors[i],
                line = dict(width=0.5, color='rgb(40,40,40)'),
                sizemode = 'area',
                opacity = 1
            ),
            name = '{}'.format(city_read),
            legendgroup = '{}'.format(city_read)
        )
        data.append(center_of_mass)

        # Add a clear map to show port of unlading
        un_port = dict(
            type = 'scattergeo',
            geo = geo_key,
            locationmode = 'world',
            lat = [un_lat],
            lon = [un_lon],
            text = '{}'.format(city_read),
            marker = dict(
                size = 10,
                color = colors[i],
                line = dict(width=0.5, color='rgb(40,40,40)'),
                sizemode = 'area'
            ),
            name = '{}'.format(city_read),
            showlegend = False
#             legendgroup = '{}'.format(city_read)
        )
        data.append(un_port)

        # One more: add lines connecting ports of unlading
        # with their respective centers of mass
        line = dict(
            geo = 'geo',
            type = 'scattergeo',
            locationmode = 'world',
            lon = [un_lon, lon_avg],
            lat = [un_lat, lat_avg],
            mode = 'lines',
            name = city_read,
            showlegend = True,
            legendgroup = '{}'.format(city_read),
            line = dict(
                width = 1,
                color = colors[i],
            )
        )
        data.append(line)
        
    return data, layout

In [23]:
title = '<b>Average Lat-Lon of Ports of Lading <br />\
Presented by Ports of Unlading: 2017</b>'

data, layout = plotly_avg_coords(city_counts_dfs, title)
        
fig = { 'data':data, 'layout':layout }
py.iplot( fig, filename='CofM Ports of Lading', height=600, width=1200 )

Ok, that took longer than I intended to format a new map, but it looks like a useful summary of the places from which different cities receive imports.  Tacoma, Long Beach, and LA all pretty solidly receive East Asian imports, overwhelming everything else.  But Seattle and Houston have as much stuff coming from East and West that the average longitude of a container coming in is pretty close to zero (Greenwich).  Just to see how stable this pattern is, let's compare to the earliest year for which Enigma has these data: 2014.

When I went back to the relevant page, the difference is striking; while there are far fewer records for that year (the whole db is 11 million, as opposed to the ~28 million rows for the 2017 database), the top 6 ports of unlading account for a much larger proportion of overall traffic:

    Frequency Rank for Port of Unlading in 2014:
    Los Angeles -  2,025,070
    Long Beach  -  1,670,075
    NYC/Newark  -  1,178,130
    Tacoma      -    908,218
    Houston     -    676,386
    Total rows  - 11,025,607
    
That means that Los Angeles handled 5% of the cargo in 2017 compared to what it received in 2014?  That doesn't seem right.  [This page](https://www.portoflosangeles.org/maritime/stats.asp) of annual traffic to the Port of Los Angeles contradicts Enigma's data, showing a gradual but consistent *increase* in containers (twenty-foot-equivalent units) handled by that port over the last decade, and indeed going back to the 80s.  Their monthly breakdown shows over 4 million inbound loaded TEUs for the last few years.  [Long Beach's site](http://www.polb.com/economics/stats/yearly_teus.asp) similarly lists pretty consistent volumes, between 3 and 4 million every year.  Obviously Enigma's data, while extensive, aren't comprehensive or authoritative.  Let's check the rest of the years for Enigma's data sets:

    Frequency Rank for Port of Unlading in 2015:
    Los Angeles -    291,567
    Long Beach  -    237,690
    NYC/Newark  -    211,192
    Seattle     -    149,245
    Tacoma      -    149,245
    Total rows  -  1,914,262
    
    Frequency Rank for Port of Unlading in 2016:
    Los Angeles -  5,024,788
    NYC/Newark  -  4,468,536
    Long Beach  -  3,979,732
    Tacoma      -  2,272,418
    Savannah    -  1,827,115
    Total rows  - 29,209,522
    
Obviously, the datasets are highly variable.  The top ports of unlading across the years are pretty consistent, but the total data present are highly variable, and the number listed for each US port similarly fluctuate.  For a less biased look, [the Census Bureau](https://www.census.gov/foreign-trade/index.html) claims to be the official source of statistics on imports and exports in the US.  The [International Trade Administration](https://www.trade.gov/mas/ian/tradestatistics/index.asp) also keeps some data.  Their [latest summary](https://www.trade.gov/mas/ian/build/groups/public/@tg_ian/documents/webcontent/tg_ian_005537.pdf), for 2016, lists total imports for the US at 2.7 trillion USD that year.

BTW, while looking into getting more complete access to summary statistics for Enigma's datasets, I came across [this page](https://docs.enigma.com/public/public_v20_user_python.html), which looks like the most accessible way to get programmatic access to the data.

I'll run a brief trial of the [programmatic access](https://docs.enigma.com/public/public_v20_user_python.html) below, since the 2014 data set was too large to actually download.  I'd like to see whether there's a way to query the db without actually loading it into local memory.

In [ ]:
import requests

enigma_api = 'Bearer {}'.format(credentials['enigma']['api_key'])
headers = {'authorization': enigma_api}
base_url = "https://public.enigma.com/api/"

def print_top_level_collections():
    url = base_url + "collections/"
    r = requests.get(url, headers=headers)
    collections = r.json()
    for collection in collections:
        print(collection['display_name'])
        print(collection['description_short'])
        print(collection['id'] + "\n")
        
def find_current_snapshot_id(dataset_id):
    url = base_url + "datasets/" + dataset_id
    r = requests.get(url, headers=headers)
    dataset = r.json()
    return dataset['current_snapshot']['id']

def snapshots_in_collection(collection_id):
    url = base_url + "datasets/?parent_collection_id=" + collection_id
    r = requests.get(url, headers=headers)
    datasets = r.json()
    return [
        dataset['current_snapshot']['id']
        for dataset in datasets
            if dataset['current_snapshot'] is not None
        ]

def get_matching_rows(snapshot_id, search_string):
    url = base_url + "snapshots/" + snapshot_id + "?row_limit=1000&query=" + search_string
    r = requests.get(url, headers=headers)
    snapshot = r.json()
    return snapshot['table_rows']['rows']

def get_matching_rows(snapshot_id, search_string, field_index):
    url = base_url + "snapshots/" + snapshot_id + "?row_limit=1000&query=" + search_string
    r = requests.get(url, headers=headers)
    snapshot = r.json()
    rows = snapshot['table_rows']['rows']
    return [
        row
        for row in rows if search_string in row[field_index].lower()
        ]

def get_dataset_ids_in_collection(collection_id, max):
    url = base_url + "datasets/?parent_collection_id=" + collection_id
    headers['Range'] = 'resources=%d-%d' % (0, max - 1)
    r = requests.get(url, headers = headers)
    datasets = r.json()
    return [
        dataset['id']
        for dataset in datasets
        ]

def get_all_datasets_in_collection(collection_id):
    url = base_url + "datasets/?parent_collection_id=" + collection_id
    r = requests.head(url, headers = headers)
    num_datasets = int(r.headers.get('content-range').split("/")[1])
    dataset_ids = []
    for start in range(0, num_datasets, 10):
        headers['Range'] = 'resources=%d-%d' % (start, start + 9)
        r = requests.get(url, headers=headers)
        datasets = r.json()
        for dataset in datasets:
            dataset_ids.append(dataset['id'])
    return dataset_ids
        
# print_top_level_collections()

In [ ]:
bill_of_lading_summary_id = '057fee6f-ec01-47c5-990d-02c1e35aaedc'
bills_of_lading_2014_id = '69806aa4-3090-4b2c-878c-8231f3791f17'
# snapshots_in_collection(bills_of_lading_2014_id)

get_all_datasets_in_collection(bill_of_lading_summary_id)

In [ ]:
import io

dataset_id = '69806aa4-3090-4b2c-878c-8231f3791f17'
snapshot_id = find_current_snapshot_id(dataset_id)
search_string = 'Seattle'
cols = ['foreign_port_of_lading']
url = base_url + "snapshots/" + snapshot_id + "?row_limit=10000&query=" + search_string
# url = base_url + "datasets/?parent_collection_id=" + dataset_id + "?row_limit=1000000&query=" + search_string
response = requests.get(url, headers=headers).content
# seattle_shipping_2014 = pd.read_csv(io.StringIO(response.decode('utf-8')), usecols=cols)
# df.head()
# response[:100]
print(url)

The cell above contains the indicated structure for retrieving rows from the 2014 Bills of Lading collection, but if I try substituting "1000000" into the 'row_limit' string, it returns the response that their exploratory mode is limited to 10000 (ten thousand) records at a time.  Therefore there doesn't appear to be a programmatic way to retrieve a large subset of the data set using a limiting criterion.  I'll instead return to just downloading csv files and loading them.  Given the huge sizes involved, however, I'll stick to just comparing imports to Seattle over the years available.

## Seattle Imports Across the Years

The respective number of rows matching Seattle as the port of unlading for each year are:

    2014 - 492,215
    2015 - 149,267
    2016 - 923,437
    2017 -  27,100
    
Their csvs, once downloaded, occupy [302, 95, 523, 15] MBs, respectively, on the local disk.

In [20]:
seattle_csvs = []
for i in range(4, 8):
    seattle_csvs.append('seattle_shipping_201{}_csv'.format(i))
    
seattle_lading_years = {}
for file in seattle_csvs:
    year = file.split(sep='_')[2]
    seattle_lading_years[year] = pd.read_csv(paths[file], 
                                             usecols=['foreign_port_of_lading'])

In [21]:
seattle_lading_years['2014'].head()

,foreign_port_of_lading
0,"Shanghai ,China (Mainland)"
1,"Tanjung Pelepas,Malaysia"
2,"Shanghai ,China (Mainland)"
3,"Shanghai ,China (Mainland)"
4,"Pusan,South Korea"


First, some preliminary cleaning, before I can grab coords for each city.  Need to drop any NaN, and identify any 5-digit codes in place of city names.

In [142]:
for year, series in seattle_lading_years.items():
    print(series.isnull().sum())

foreign_port_of_lading    0
dtype: int64
foreign_port_of_lading    0
dtype: int64
foreign_port_of_lading    0
dtype: int64
foreign_port_of_lading    0
dtype: int64


In [23]:
seattle_lading_years['2014'].dropna(inplace=True)

In [26]:
code_matches = code_lookup(seattle_lading_years)

2014:
Series([], Name: foreign_port_of_lading, dtype: int64)
2015:
57045    2026
57073     101
35136       2
57019       1
Name: foreign_port_of_lading, dtype: int64
2016:
57073    2214
57045     173
53304      38
35136      29
99999       5
35180       5
57037       3
55225       1
Name: foreign_port_of_lading, dtype: int64
2017:
57037    721
57073      2
35136      1
Name: foreign_port_of_lading, dtype: int64


In [27]:
print(code_matches)

{'99999', '53304', '57037', '35136', '55225', '57019', '57045', '35180', '57073'}


In [28]:
origin_codes_dict = {'57073': 'Nansha, China (Mainland)', 
                     '53304': 'Pipavav, India', 
                     '30110': 'Covenas, Columbia', 
                     '57037': 'Yangshan, China (Mainland)'
                    }
for code in code_matches:
    if code not in origin_codes_dict.keys():
        print(code)

99999
35136
55225
57019
57045
35180


That yielded just one new hit, when comparing to [these](https://www.cbp.gov/sites/default/files/assets/documents/2017-Feb/appendix_f_0.pdf) [two](http://customscodes.com/foreign_port_code.php) resources:

In [29]:
origin_codes_dict[57019] = 'Zhenjiang, China'

Let's try both of the graph types above: I'll start with the averaged lat/lon approach.  First I need to clean up the port names and retrieve lat/lon for them, however:

In [155]:
%%time

# cities_2017_dfs = import_shipping_data(cities_2017_paths)
# seattle_lading_coded = replace_city_codes(seattle_lading_years, origin_codes_dict)
# seattle_lading_formatted = format_ports(seattle_lading_coded)

# lat_long, failures = geolocate(seattle_lading_formatted)

# seattle_lading_coords = get_coords(seattle_lading_formatted, lat_long)
# seattle_lading_counts = get_counts(seattle_lading_coords)

2014        
Coordinates couldn't be found for  1317 ports of lading; 0.27% of all records
2015        
Coordinates couldn't be found for   903 ports of lading; 0.61% of all records
2016        
Coordinates couldn't be found for  1615 ports of lading; 0.17% of all records
2017        
Coordinates couldn't be found for     1 ports of lading; 0.00% of all records
Wall time: 10.7 s


In [152]:
seattle_lading_counts['2017'].head()

,city,lat_long,lat,long,containers,text
0,"Hong Kong, Hong Kong","(22.396428, 114.109497)",22.396428,114.109497,940,"Hong Kong, Hong Kong: 940 container(s)"
3,"Tsingtao, China","(36.067108, 120.382609)",36.067108,120.382609,23,"Tsingtao, China: 23 container(s)"
4,"Pusan, South Korea","(35.1795543, 129.0756416)",35.179554,129.075642,3240,"Pusan, South Korea: 3240 container(s)"
5,"Yantian, China","(22.556499, 114.236875)",22.556499,114.236875,999,"Yantian, China: 999 container(s)"
11,"Shanghai, China","(31.2303904, 121.4737021)",31.230390,121.473702,2936,"Shanghai, China: 2936 container(s)"


In [157]:
print(failures)

['Port Redon, Vietnam', 'High Seas, United States Off Shore Tanker Transshipment Points', 'Pursan, Turkey', 'Nagoya Ko, Japan', 'Puduchcheri, India', 'All Other Southern Asia, N E C Ports, Southern Asia, N E C', 'All Other Oman Ports, Oman']


Ok, that looks good.  I'll save the counts, and the dict.

In [160]:
with open(paths['lat_long_dict_pkl'], 'rb') as f:
    old_dict = pickle.load(f)
    
print(len(old_dict.keys()), len(lat_long.keys()))
new_dict = old_dict
for city, coords in lat_long.items():
    if city not in old_dict.keys():
        new_dict[city] = coords
        
print(len(new_dict))

293 245
362


In [161]:
paths['seattle_annual_pkl'] = os.path.join(paths['data_dir'], 'seattle_annual.pkl')

with open(paths['lat_long_dict_pkl'], 'wb') as f:
    pickle.dump(new_dict, f)
with open(paths['seattle_annual_pkl'], 'wb') as f:
    pickle.dump(seattle_lading_counts, f)

In [3]:
with open(paths['seattle_annual_pkl'], 'rb') as f:
    seattle_lading_counts = pickle.load(f)

Now, to check the plots for multiple years of Seattle bills of lading.

In [8]:
for year in seattle_lading_counts.keys():
    name_translator[year] = 'Seattle, Washington'

In [24]:
title = '<b>Average Lat-Lon of Ports of Lading <br />\
Bound for Seattle: 2014-2017</b>'

data, layout = plotly_avg_coords(seattle_lading_counts, title, scale=5E3)
        
fig = { 'data':data, 'layout':layout }
py.iplot( fig, filename='CofM Ports of Lading', height=600, width=1200 )

Ok, so, for Seattle at least, the average number of containers coming from ports didn't shift very far East or West for the years 2014-2016.  The data are highly variable for those years; many more records from 2016 than 2014 (and note that I had to change the scale to make the markers more visible, compared to the 2017 data mapped earlier).  2017 has far fewer records for shipments to Seattle, and therefore it's less surprising that the geographic center of their distribution also shifted.  This almost certainly represents bias in sampling in the data, rather than an actual shift in the number and origin points of cargo received; as indicated above, Enigma's data are highly variable for other ports, and in ways that don't line up with the largest ports' own records.

I found [some historical data](https://www.portseattle.org/About/Publications/Statistics/Seaport/Pages/10-Year-History.aspx) for the Port of Seattle, and there is indeed significant variety over the years indicated, but container imports between 500K and 1 million are actually believable.  For some reason the data on that page don't cover more recent years; the [most recent complete report](https://www.nwseaportalliance.com/sites/default/files/nwsa_annualreport_2016_full_0.pdf) I could find lists 3.6 million total TEUs, but doesn't break them down by import full/empty vs export full/empty.

For the hell of it, let's look at the city-by-city breakdown of ports of lading for Seattle by year, too:

In [34]:
title = '<b>Ports of Lading for Containers Received in Seattle, <br />\
Presented by Year: 2014-2017</b>'

data, layout = plotly_subplots(seattle_lading_counts, scale=50, title=title, rows=2)

fig = { 'data':data, 'layout':layout }
py.iplot( fig, filename='Ports of Lading', height=900, width=1000 )

Ok, not too surprising; the breakdown looks pretty much the same for most years; mostly East Asia, but also good representation of the E.U. and a smattering of Latin America.  Importantly, the phenomenon of a ton of cargo coming via British Columbia has existed for all of those years.

I'd say that's enough for enumerating ports of lading for containers.  Let's move on.

In [ ]:
seattle_lading_counts = {}
for year, df in seattle_lading_coords.items():
    seattle_lading_counts[year] = df.value_counts()

In [ ]:
seattle_lading_coords['2017'].groupby(['foreign_port_of_lading']).head()

In [ ]:
data, layout = plotly_avg_coords(city_counts_dfs, title)
        
fig = { 'data':data, 'layout':layout }
py.iplot( fig, filename='CofM Ports of Lading', height=600, width=1200 )

In fact, at the risk of spending too much time learning another mapping paradigm in plotly, let's see if I can create such a summary.  I'll just use the data on hand, rather than all 27 million records for 2017, at this point.  I don't know if it'll be easy to overlay a heatmap onto an actual geo world map.  One alternative is to do a [choropleth map](https://plot.ly/python/choropleth-maps/), but I'd want really small units demarcating areas, so the resolution of the shipping data could be displayed effectively.  If I want to try that, there's [precedent](https://community.plot.ly/t/create-your-own-choropleth-map-with-custom-shapefiles/2567/4) for how to feed in custom boundaries, and a data source for getting higher resolution areas than just national borders is [here](http://devecondata.blogspot.com/2007/09/administrative-boundaries.html).  I'll download the multi-level shapefile to the 'data_dir' path.

As [this discussion](https://community.plot.ly/t/what-is-the-best-way-to-put-a-heatmap-on-a-map/3661/2) points out, another approach might be to use explicit markers like above, but make their opacity dependent upon the number of overlapping points represented.  An example that they cite, which looks pretty good, is [here](https://i.stack.imgur.com/9ek6k.png).  Of course, in my case, the cities' lat/long coordinates are all identical, so you'd end up with tiny dots that just get slightly darker, which wouldn't give the exact impression I'm going for.  You could, however, use a function from either numpy or python's built-in random module to introduce uniformly distributed permutation of the lat-long coords based on how many records come from a given port, in order to spread the effect out spatially.  But I don't think I could contain the effect based on shorelines, meaning it would bleed over into surrounding water.  That might be ok, but it does sound like quite a bit of fiddling to set it up.  One additional idea would be to specify a different color along a range of colors available on a yellow-orange-red spectrum that the markers could assume, based on the total number of containers coming from that city.  But that might wash out the original message of the opacity-based approach.

...Ah, let's just try it.

In [ ]:
# Make a new, combined df of ports of lading
# Keeping only city name, identifier, lat, and long
frames = []
columns = ('container_number', 'foreign_port_of_lading', 'lat_long')
for city, df in cities_2017_dfs.items():
    frames.append(df.loc[:, columns])
top_6_df = pd.concat(frames)
top_6_df.sort_values(by='foreign_port_of_lading', inplace=True)
print('top_6_df.shape: {}'.format(top_6_df.shape))

# Specify a dict linking coords and total counts
all_counts = top_6_df['lat_long'].value_counts().to_dict()
print('\nCounts for first 5 cities in top_6_df:')
for k, v in {k: all_counts[k] for k in sorted(all_counts.keys())[:5]}.items():
    print('{:<35} {}'.format(str(k), v))
top_6_df.head()

In [ ]:
top_6_per = top_6_df
max_count = top_6_df['foreign_port_of_lading'].value_counts().max()
factor=1

# Break out lat/long into separate columns, with noise 
# based on how highly that city's count ranks
top_6_per['lat'] = top_6_per['lat_long'].apply(
    lambda x: (x)[0]+np.random.normal(
        loc=0, scale=all_counts[x]*factor/max_count))
top_6_per['long'] = top_6_per['lat_long'].apply(
    lambda x: (x)[1]+np.random.normal(
        loc=0, scale=all_counts[x]*factor/max_count))

top_6_per.head()

The first time I tried running this, plotly's server returned a friendly warning that there are too many data points, and that it bogs down with more than 40,000 points in scatter mode.  I have about 10x that.  To get a look at whether or not the formatting makes sense, let's try just running with a subset.  I'll choose cities with the max, min, and median total exports, and see how they look.

In [ ]:
max_count = top_6_df['foreign_port_of_lading'].value_counts().max()
min_count = top_6_df['foreign_port_of_lading'].value_counts().min()
med_count = top_6_df['foreign_port_of_lading'].value_counts().median()
subset_index = []
for k, v in all_counts.items():
    if v in [max_count, min_count, med_count]:
        subset_index.append(k)
        
subset = top_6_per.set_index(['lat_long']).loc[subset_index]
subset.shape

In [ ]:
tools.set_credentials_file(username=plotly_username, api_key=plotly_api_key)

top_6_plus = cities_2017_dfs
top_6_plus.update(top_6=top_6_df)

colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","lightgrey"]
cities = []
markersize = 3
scale = 3
log_scale = 100

city = dict(
    type = 'scattergeo',
    locationmode = 'ISO-3',
#     lon = top_6_per['long'],
#     lat = top_6_per['lat'],
#     text = top_6_per['foreign_port_of_lading'],
    lon = subset['long'],
    lat = subset['lat'],
    text = subset['foreign_port_of_lading'],
    marker = dict(
        size = 5,
        color = 'rgb(100, 100, 100, 0.1)',
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'
    )
)
cities.append(city)

layout = dict(
        title = 'Ports of Lading Received by top 6 US Ports',
        showlegend = True,
        geo = dict(
            scope='world',
            projection=dict( type='Mercator' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict( data=cities, layout=layout )
py.iplot( fig, validate=False, filename='top6_cities_heatmap' )

Ok, the effect is still too much for that library.  There are 54,000 data points in the reduced subset.  Furthermore, when I zoom in on the max city (Shanghai), it looks like a large cluster of dots that're so uniformly spread out that it's pretty equivalent to just plotting a single marker of much greater area than the others (i.e., the approach I took when originally plotting these ports).  Clearly, this approach isn't going to generate the results I'm looking for.  The suggested adjustments they have are:

    Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
    (1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
    (2) Trying using the image API to return an image instead of a graph URL
    (3) Use matplotlib
    (4) See if you can create your visualization with fewer data points
    
Another possibility that I just hit upon is to try out [gmplot](https://github.com/vgm64/gmplot) a package that wraps the Google Maps API for Python, and says that it can overlay heatmaps fairly easily.  But it looks like an individual contributor's project, and the documentation for it is almost nonexistent.

At this point, however, I'm more inclined to just attempt the choropleth approach.  I unzipped the shapefiles from gadm; there are 6 ('gadm_admX', where X is 0 through 5).  Their sizes vary between ~50MB and 500MB, but not monotonically increasing or decreasing.  The source page said the zip contained multiple levels of complexity and resolution, so I'm assuming that each file will represent the whole world, but at different levels of precision, rather than having some files only cover certain geographical regions.  I'll try the smallest file first, and see if I can get it to plot as polygons.

In [ ]:
import geopandas as gpd

geo_s = {}
for i in range(6):
    geo_s['geodf'+str(i)] = gpd.read_file(paths['gadm28_adm{}_shp'.format(i)])

In [ ]:
for k, v in geo_s.items():
    print(k, type(v))
    
for k, v in geo_s.items():
    print(k, v.shape)
geo_s['geodf0'].head()

Ok, clearly my early supposition was incorrect.  Instead, the data for all countries *are* distributed across multiple shapefiles, and it's unclear whether there's any redundancy (i.e., subregions being proper subsets of regions covered as polygons elsewhere in the data set).  The titles of columns vary somewhat between shapefiles, but the fourth column (3 in zero-index-speak) is always the overall country's name.

In [ ]:
# shape_counts = geo_s['geodf5'].iloc[:, 3].value_counts()
# shape_counts
for k, v in geo_s.items():
    print('{}:\n{}\n\n'.format(k, v.iloc[:, 3].value_counts()))

In [ ]:
for k, v in geo_s.items():
    print('{}: {}'.format(k, v.iloc[:,3].unique().shape))

Ok, I'm starting to get it.  The `geodf0` is the most comprehensive, but just contains national boundaries.  `geodf1` appears to contain state/province boundaries, since the U.S. has 52, and the U.K. has 4 (I'm guessing England, Wales, Scotland, and Northern Ireland), but strangely Macedonia has >80 entries here.  Obviously this level of complexity of boundaries isn't the same for all countries.

`geodf2` is getting too complicated: 3,148 entries for the US (probably county level).  Probably `geodf3` through 5 are just more extravagently labeled subsets of even fewer countries.  It's kind of funny that the last file only contains France and Rwanda, but has >10,000 entries for each.  How can you even split France into 36,000 polygons?

Anyways, I definitely just `geodf0`, or perhaps `geodf1`.  The latter might be overkill, but they contain a tabulation of their areas; perhaps I could just keep the largest, and use national boundaries for everything else.

It's a bit of a uphill slog, but I should check [these](https://plot.ly/python/county-choropleth/) [references](https://plot.ly/~jackp/18273.embed).

As an alternative to plotly, it's [been pointed out](http://geopandas.org/mapping.html) that geopandas has a built-in module to output via matplotlib.

In [ ]:
%%time

import matplotlib.pyplot as plt
if ax:
    ax.clear()

f, ax = plt.subplots(1, figsize=(15, 10))
ax = geo_s['geodf0'].plot(ax=ax)
lims = plt.axis('equal')
plt.show()

Wow, that takes a long time to render, even with the fewest polygons possible (i.e., national boundaries).  I wonder if I can simply ignore any other than those that export to the US?

In [ ]:
countries = top_6_df['foreign_port_of_lading'].apply(lambda x: x.split(sep=', ')[-1])
countries.value_counts().shape

That's 82 instead of 256 countries.  Let's see if it speeds things up, and how it looks:

In [ ]:
# subset = pd.merge(geo_s['geodf0'], left_on='NAME_ENGLI', countries, how='right')
geodf0 = geo_s['geodf0']
subset = geodf0[geodf0['NAME_ENGLI'].isin(countries.tolist())]
print(subset.shape)
subset.head()

In [ ]:
%%time
if ax:
    ax.clear()

f, ax = plt.subplots(1, figsize=(15, 10))
ax = subset.plot(ax=ax)
lims = plt.axis('equal')
plt.show()

In [ ]:
geodf1 = geo_s['geodf1']
geodf1.head()
subset2 = geodf1[geodf1['NAME_0'].isin(countries.tolist())]
print(subset2.shape)

Eh, that's still going to be too many to plot.  How about just trying to find the smaller polygons that contain a lat-lon point within the top_6_df list?  [Here's a write-up](https://gis.stackexchange.com/a/208574) about options for doing that.

In [ ]:
import geopandas
point = geopandas.GeoDataFrame(top_6_df['lat_long'])
poly  = geopandas.GeoDataFrame(geodf1['geometry'])
from geopandas.tools import sjoin
pointInPolys = sjoin(point, poly, how='left')
grouped = pointInPolys.groupby('index_right')
list(grouped)

In [ ]:
# from geopandas.tools import sjoin
# countries = geopandas
# from shapely.geometry import Polygon, Point, MultiPolygon
# points = Point(top_6_df['lat_long'])
points = gpd.GeoDataFrame(top_6_df['lat_long'])
# poly = 
crs = {'init': 'epsg:4326'}
# gdf = gpd.GeoDataFrame(points, crs=crs, geometry=geometry)
pointInPolys = gpd.tools.sjoin(points, geodf1['geometry'], how='left')
grouped = pointInPolys.groupby('index_right')
list(grouped)

In [ ]:
dir(geopandas)

In [ ]:
countries.unique().tolist()

In [ ]:
data = [ dict(
        type='choropleth',
#         colorscale = scl,
        autocolorscale = False,
        locations = geo_s['geodf0']['geometry'].exterior,
#         z = shp_brabant_lim['Color'].astype(float),
        text = geo_s['geodf0'].iloc[:,3],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Colorscale")
        ) ]


layout = dict(
    height=600,
    autosize=True,
    hovermode='closest'#,
#     mapbox=dict(
#         layers=[
#             dict(
#                 sourcetype = 'geojson',
#                 source = 'DIRECTORY/shp_brabant_lim.json',
#                 type = 'fill',
#                 color = 'rgba(163,22,19,0.8)'
#             )
#         ],
#         accesstoken=mapbox_access_token,
#         bearing=0,
#         center=dict(
#             lat=51.6978,
#             lon=-5.3037
#         ),
#         pitch=0,
#         zoom=5.2,
#         style='light'
#     ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='National_boundaries')

In [ ]:
dir(geo_s['geodf0'].iloc[0,-1])
geo_s['geodf0'].iloc[0,-1].exterior.simplify(0.1)
# geo_s['geodf0'].iloc[:,-1].exterior

In [ ]:
geo_s['geodf2'].iloc[:,3].value_counts()['United States']

## Part 2: Checking Weights

Visualizing the number of records associated with different cities shipping to Seattle is cool, but the quantity and quality of goods might not be well represented just by number of containers.

A more nuanced look would involve checking the total volume of goods coming from these places, in which case you'd probably want raw tonnage, which might be extractable from the data set.

In [35]:
print('Missing data in harmonized_weight_unit')
for city, df in cities_2017_dfs.items():
    missing_rows = df['harmonized_weight_unit'].isnull().sum()
    total_rows = df.shape[0]
    print('{:>12}: {:>7,} missing fields in {:>7,} rows ({:>4}%).'.format(
        city, missing_rows, total_rows, np.round(missing_rows/total_rows*100, 1)))
    
print('\nMissing data in harmonized_value')
for city, df in cities_2017_dfs.items():
    missing_rows = df['harmonized_weight'].isnull().sum()
    total_rows = df.shape[0]
    print('{:>12}: {:>7,} missing fields in {:>7,} rows ({:>4}%).'.format(
        city, missing_rows, total_rows, np.round(missing_rows/total_rows*100, 1)))

print('\nMissing data in description_text')
for city, df in cities_2017_dfs.items():
    missing_rows = df['description_text'].isnull().sum()
    total_rows = df.shape[0]
    print('{:>12}: {:>7,} missing fields in {:>7,} rows ({:>4}%).'.format(
        city, missing_rows, total_rows, np.round(missing_rows/total_rows*100, 1)))

Missing data in harmonized_weight_unit
     Houston:  29,547 missing fields in  39,017 rows (75.7%).
  Long_Beach: 100,242 missing fields in 116,721 rows (85.9%).
 Los_Angeles:  91,704 missing fields in 104,008 rows (88.2%).
  NYC_Newark:  73,111 missing fields in  94,148 rows (77.7%).
     Seattle:  19,172 missing fields in  27,098 rows (70.8%).
      Tacoma:  27,512 missing fields in  33,920 rows (81.1%).

Missing data in harmonized_value
     Houston:  21,647 missing fields in  39,017 rows (55.5%).
  Long_Beach:  90,026 missing fields in 116,721 rows (77.1%).
 Los_Angeles:  68,671 missing fields in 104,008 rows (66.0%).
  NYC_Newark:  64,691 missing fields in  94,148 rows (68.7%).
     Seattle:  10,662 missing fields in  27,098 rows (39.3%).
      Tacoma:  13,825 missing fields in  33,920 rows (40.8%).

Missing data in description_text
     Houston:       0 missing fields in  39,017 rows ( 0.0%).
  Long_Beach:       0 missing fields in 116,721 rows ( 0.0%).
 Los_Angeles:       0 mis

Or they might not.  There are a lot of missing data for weight of containers; many others have a weight of zero listed.  Obviously the data entry is not standardized and strictly enforced.  But some of the extended text descriptions include some accounting of weight:

In [36]:
for desc in cities_2017_dfs['Seattle'].loc[::1000, 'description_text'].iteritems():
    print(desc, '\n')

# sea2017_df.loc[::10, 'description_text']

(0, 'SOFA & UPHOLSTERY PO418287/418285/418967 "SH "') 

(1000, 'WELDING WIRE') 

(2000, '55  CTN GDSM SOFA___TEL_ 1-650-685 8808 FAX_ 1-650-685 8898 S/C_US0002D2X CY TO CY 1X40_HQFCL') 

(3000, "LADIES' 100% LINEN KNITTED  CA RDIGAN") 

(4000, "CHEMICAL WOOD PULP, SODA OR SULPHATE, OTHER T HAN D - 'THUNDER BAY' FULLY BLEACHED HARDWOOD KRAFT PULP") 

(5000, 'ROLLS') 

(6000, 'TABLEWARE AND KITCHENWARE OF PLASTICS PLASTIC KITCHEN WARE 669317 GROUP PN3 +1 817-442-818 7') 

(7001, "LOADED INTO 1 40'HIGH CUBE CONTAINER GRODAN / MINERAL WOOL NET WEIGHT :4952.0000 KGS HS-NO 68061000") 

(8001, 'ORDER  0085910 ORIENTED STRAND BOARD SUMMARY ID  SUM2092 FREIGHT PREPAID HARMONIZATION COD ES  4410.12.00 NET UNIT  196.458 MSFT') 

(9001, 'CALCIUM STEARATE   HS CODE : 3 812.30   8255556   UDAB   SEA WAYBILL   FREIGHT COLLECT') 

(10001, 'DRIED LEGUMINOUS VEGETABLES, SHELLED, WHETHER OR N - X 100 LB BAGS CROP YEAR 2017: US CHIC KPEAS 2 X20 FCLCROP YEAR 2017: US CHICKPEAS 1 090 X 100 LB BAGS HS CODE: 

I count about 5 rows with weight info among those 28 that I sampled.  They're pretty variable, but sense could be made of them.  I wonder whether the rows that lack values in the dedicated weight column are more likely to contain weight information in the text description.

But identifying the weight data from such a nonstandard format is tricky.  I might just need to pass a series of key terms like 'WT' and 'WEIGHT' with OR operators in between to a re pattern, or I may want to try sending the strings through some NLTK function.

In [60]:
import re

prefixes = ['NET', 'GROSS', 'METRIC']
triggers = ['WT', 'WEIGHT']
units = ['POUNDS', 'LBS', 'TONS', 'KG', 'KGS']
strings = [*prefixes, *triggers, *units]

weight_words = ['NET', 'GROSS', 'METRIC', 'WT', 'WEIGHT', 'POUNDS',
               'POUND', 'LBS', 'LB', 'TONS', 'TONNES', 'TON', 'KG', 'KGS']

weight_words_string = '|'.join(x for x in weight_words)
# pattern = '('+weight_words_string+'+[:?]\s{1,2}\d+)'
# pattern = '(('+weight_words_string+')[:?]\s{1,2}[\d.,]+\s{1,2}('+weight_words_string+'))'
pattern = '('+weight_words_string+')'
pattern = re.compile(pattern)
pattern2 = re.compile('\w+\s\d+')

results = {}
df = cities_2017_dfs['Seattle']
df2 = df[df['harmonized_weight'].isnull()]# | df['harmonized_weight']==0]
df3 = df[df['harmonized_weight'] == 0]
df4 = pd.concat([df2, df3], axis=0)
df5 = df4[~df4.index.duplicated(keep='first')]
weight_matches = df5[df5['description_text'].str.match(pattern2)]
# for i, v in df2.iteritems():
#     if re.search(pattern, v):
#         matches = re.search(pattern, v).groups()#[0]
#         results[i] = matches
# weight_matches = pd.Series(data=results)
df2.shape, df3.shape

((10662, 35), (8757, 35))

In [66]:
# weight_matches[12000]
# re.findall(r'[\d,]+[.]?[\d,]+', sea2017_df.loc[12000, 'description_text'])
# sea2017_df.loc[12000, 'description_text']
# weight_matches.head()
weight_matches.shape
# df5.shape
df5.head()
# cities_2017_dfs['Seattle'].head()
df = cities_2017_dfs['Seattle']
df[df.loc[:, 'harmonized_value'].notnull()].head()

,identifier,trade_update_date,run_date,vessel_name,port_of_unlading,estimated_arrival_date,city,record_status_indicator,place_of_receipt,port_of_destination,...,piece_count,description_text,harmonized_number,harmonized_value,harmonized_weight,harmonized_weight_unit,lat,long,lat_long,containers
13,201712262236,2017-11-28T00:00:00,2017-12-26T00:00:00,NEW YORK EXPRESS,"Seattle, Washington",2017-12-21T00:00:00,"Yantian, China",New,HAIPHONG,NaN,...,1243,"APPAREL, MEN S WOVEN SHIRT, LADIES STRIPES SH",620520.0,241660.0,12083.0,Kilograms,22.556499,114.236875,"(22.556499, 114.236875)",999
29,201712263368,2017-12-01T00:00:00,2017-12-26T00:00:00,NEW YORK EXPRESS,"Seattle, Washington",2017-12-21T00:00:00,"Pusan, South Korea",New,BUSAN,NaN,...,19,EDTA-ACID,NaN,0.0,19057.0,Kilograms,35.179554,129.075642,"(35.1795543, 129.0756416)",3240
30,201712263445,2017-12-01T00:00:00,2017-12-26T00:00:00,NEW YORK EXPRESS,"Seattle, Washington",2017-12-21T00:00:00,"Yangshan, China",New,"SHANGHAI, SHANGHA",NaN,...,640,OFFICE CHAIR,940130.0,215008.0,10750.0,Kilograms,24.466675,112.626495,"(24.466675, 112.626495)",721
31,201712263500,2017-12-01T00:00:00,2017-12-26T00:00:00,NEW YORK EXPRESS,"Seattle, Washington",2017-12-21T00:00:00,"Yangshan, China",New,"SHANGHAI, SHANGHA",NaN,...,54,NAILHEADS HTS CODE 940161 PO 12030814562 IF A,940161.0,57708.0,2885.0,Kilograms,24.466675,112.626495,"(24.466675, 112.626495)",721
32,201712263681,2017-12-15T00:00:00,2017-12-26T00:00:00,NEW YORK EXPRESS,"Seattle, Washington",2017-12-21T00:00:00,"Pusan, South Korea",New,"TIANJIN XINGANG,",3512,...,19,EDTA-ACID S/C S17ASC112 MR CODE BLUBRE1 XFAX +...,292249.0,0.0,0.0,NaN,35.179554,129.075642,"(35.1795543, 129.0756416)",3240


Ok, so that list of words for capturing descriptions that might mention the weight of the cargo yields results that are a little underwhelming: only ~7,000 of the 19,000 rows that have no weight listed.

In [ ]:
results = {}
for i, v in empty['description_text'].iteritems():
    if re.search(pattern, v):
        matches = re.search(pattern, v).groups()#[0]
        results[i] = matches
empty_weight_matches = pd.Series(data=results)

results = {}
for i, v in zeroes['description_text'].iteritems():
    if re.search(pattern, v):
        matches = re.search(pattern, v).groups()#[0]
        results[i] = matches
zeroes_weight_matches = pd.Series(data=results)

In [ ]:
empty_weight_matches.shape, zeroes_weight_matches.shape
empty_weight_matches.shape[0] + zeroes_weight_matches.shape[0]

And fewer than 5,000 of those rows happen to be in the records that lack a number in the 'harmonized_weight' column.  For now, I think it wouldn't be worth it to fine-tune the search strategy to recover those data.  Just plot the small subset of the data that include weight info.  But it wouldn't really be a great comparison to compare that plot to **all** of the data, since there may be a systematic underrepresentation of weight data from certain countries.  Actually, let's check for that:

In [ ]:
all_cities = sea2017_df['lading_formatted']\
.value_counts().rename('all_cities')

missing_weight_cities = pd.concat([
    empty['lading_formatted'].value_counts(),
    zeroes['lading_formatted'].value_counts()], axis=1)
missing_sum = missing_weight_cities.sum(axis=1).sort_values(
    ascending=False).rename('missing_weights')

cities_weights = pd.concat([all_cities, missing_sum], axis=1)
# cities_weights = missing_sum.join(all_cities)

cities_weights['ratio_missing'] = \
cities_weights['missing_weights']/cities_weights['all_cities']

In [ ]:
cities_weights.sort_values(by='all_cities', ascending=False)[:10],\
cities_weights.sort_values(by='all_cities', ascending=False)[-10:]

Ok, so the proportion of missing data is fairly high throughout, but it does look like, overall, cities that ship to Seattle less overall also have a higer tendency to fall through the cracks in terms of reporting their weight.  Let's check real quick:

In [ ]:
cities_weights.corr()

Ok, there's a weak negative correlation, meaning that cities with more records overall also have a slightly lower chance to be missing weight data.  It's not a tremendous trend, however, so it looks like summarizing the data set based on weight shouldn't be horribly misrepresentative of the total number of cities shipping.  So it might be worthwhile to graph them side-by-side after all.

In [ ]:
sea2017_df.head()

In [ ]:
sea2017_df['harmonized_weight_unit'].value_counts()

In [ ]:
re.findall('.*', 'blah')
re.findall('[\d.]+', '1234.56789')
blahp = 'LBS|POUNDS|TONS'
re.findall('(('+blahp+')?: [\d.,]+)', 'POUNDS: 234.56')
# re.findall(blahp+'\d+', 'POUNDS234.56')
# re.findall('((this|that)\'s)', 'that\'s')
sea2017_df.loc[251, 'description_text']

In [ ]:
8757 / sea2017_df.shape[0]

Probably a more interesting idea for now is to categorize *what* we're importing based on the description text.

## Analysis

One question that arises, in the case of Seattle, is why the overwhelming majority of shipments are coming from Vancouver, even though probably most of the ultimate origin of many imported goods is still China.  Vancouver is very close by, and may have a better port or otherwise have a different set of tariff/duties rules than the US, which would make it likely that foreign imports might shuttle through there.  It's a few years old, but [this article](https://www.joc.com/port-news/us-ports/us-importers-moving-more-containers-through-vancouver_20150618.html) states that many other US cities receive more imports lately from Vancouver, rather than Seattle or Tacoma, lending credence to the idea that Vancouver is a popular import point for foreign goods from overseas.  Actually, the article sites a labor dispute between longshoreman unions and ports in the PacNW as a driver for a temporary shift to Canada, which may have initiated a shift in traffic up North.  Still, the question as to why goods would pass through there on the way here seems a little puzzling.  A deeper dive into this issue could be interesting, or just too obscure to address adequately in a reasonable amount of time.